In [1]:
from retriever import create_retriever
from decodeencode import split_image_text_types
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.chat_models import ChatOllama
import sys

/home/vqa/RAG/venv8/lib/python3.8/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.1) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
__import__('pysqlite3')
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
main_folder_path = '/home/vqa/RAG/10_256'
retriever, vectorstore = create_retriever(main_folder_path)
vectorstore.get()

{'ids': ['bd93b939-03c2-11ef-a911-3335794af7e3',
  'bf0c5382-03c2-11ef-a911-3335794af7e3',
  'bf0c5381-03c2-11ef-a911-3335794af7e3',
  'bf0c5380-03c2-11ef-a911-3335794af7e3',
  'bf0c537f-03c2-11ef-a911-3335794af7e3',
  'bf0c537e-03c2-11ef-a911-3335794af7e3',
  'bf0c537d-03c2-11ef-a911-3335794af7e3',
  'bf0c537c-03c2-11ef-a911-3335794af7e3',
  'bf0c537b-03c2-11ef-a911-3335794af7e3',
  'bf0c537a-03c2-11ef-a911-3335794af7e3',
  'bf0c5383-03c2-11ef-a911-3335794af7e3',
  'bd93b938-03c2-11ef-a911-3335794af7e3',
  'bd93b937-03c2-11ef-a911-3335794af7e3',
  'bd93b936-03c2-11ef-a911-3335794af7e3',
  'bd93b935-03c2-11ef-a911-3335794af7e3',
  'bd93b934-03c2-11ef-a911-3335794af7e3',
  'bd93b933-03c2-11ef-a911-3335794af7e3',
  'bd93b932-03c2-11ef-a911-3335794af7e3',
  'bd93b931-03c2-11ef-a911-3335794af7e3',
  'c4921d2d-03c2-11ef-a911-3335794af7e3',
  'd68ab7ad-03c2-11ef-a911-3335794af7e3',
  'd68ab7ac-03c2-11ef-a911-3335794af7e3',
  'c5767894-03c2-11ef-a911-3335794af7e3',
  'c4921d33-03c2-11ef-a911-

In [4]:
def prompt_func(data_dict):
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []
    if data_dict["context"]["images"]:
        image_message = {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{data_dict['context']['images'][0]}"},}
        messages.append(image_message)
    text_message = {
        "type": "text",
        "text": (
            "Provide a precise answer to the user question based on the provided context."
            f"User question: {data_dict['question']}\n\n"
            "Context:\n"
            f"{formatted_texts}"),}

    messages.append(text_message)
    return [HumanMessage(content=messages)]

In [5]:
model = ChatOllama(model="llava")

In [6]:
# llm = ChatOllama(model="mistral")

In [7]:
chain = (
    {
        "context": retriever | RunnableLambda(split_image_text_types),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(prompt_func)
    | model
    | StrOutputParser()
)

In [8]:
# print('answer:')
# print(chain.invoke("What should I pay attention to when I use headphones with this CD Player?"))

In [9]:
# '''
# for doc in docs:
#     doc_id = doc.metadata.get('doc_id', 'Unknown ID')
#     print(f"Document ID: {doc_id}")'''

In [10]:
from datasets import Dataset
import pandas as pd
import json
from tabulate import tabulate

EVALUATION

In [11]:

data = []
with open('10_manuals.jsonl', 'r') as f:
# with open('20_manuals.jsonl', 'r') as f:
    for line in f:

        json_data = json.loads(line)
        id = json_data['id'][:-6]
        qa_data = json_data['qa_data']
        for i in qa_data:
            question = i['question']['text']
            answer = i['answer']['text']
        data.append({'id': id, 'question' : question, 'ground_truth': answer})

df = pd.DataFrame(data)
df = pd.DataFrame(data, columns = ['id', 'question', 'ground_truth'])
print(tabulate(df, headers='keys', tablefmt='psql'))

n = len(pd.unique(df['id']))
print("No.of.unique values :", n)

+-----+-------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|     | id                                                                            | question                                                                                                                        | ground_truth                                                                                                                                                                                                                                                                      

In [12]:
import pandas as pd

# Assuming your dataframe is named 'df'
# Replace 'row1' with the actual name of the column containing the values you want to group by

# Step 1: Identify unique values in row 1
# unique_values = df.iloc[0].unique()
unique_values = (pd.unique(df['id']))
print(unique_values)


# Assuming your dataframe is named 'df'
# Assuming your list of unique values is named 'unique_values_list'
# Replace 'column_name' with the actual name of the column containing the values

# Create a DataFrame with the unique values from the list
unique_values_df = pd.DataFrame({'unique_values': unique_values})

# Merge the original DataFrame with the DataFrame containing unique values
merged_df = pd.merge(df, unique_values_df, how='inner', left_on='id', right_on='unique_values')

# Group by the unique values and select the first 5 occurrences of each
df = merged_df.groupby('unique_values').head(10)

# Drop the 'unique_values' column if you don't need it in the final result
# result_df.drop(columns=['unique_values'], inplace=True)

# # Reset index of the result dataframe
# result_df.reset_index(drop=True, inplace=True)



['bose_headphones manuals_9917ef89-897c-6524-2502-2b0a91ec7d62'
 'bose_headphones_manuals_6d12e20d-cf8d-4337-b073-53d7e85e4163'
 'dell_cell phone manuals_4686e2e1-87a4-4f6a-bf20-61c646c11bb9'
 'og_wave-bma_en' 'owg_en_wms_soundlink_adapterkit'
 'samsung_audio_box_eo-sb330_um_sea_rev.1.0_140728_screen'
 'samsung_cell phone accessories manuals_de30be54-38b5-46f4-8dc4-a2376a6e571d'
 'samsung_vacuum cleaner manuals_8cb9360e-cafe-4c53-9d35-ef193667e586'
 'samsung_video gaming accessories manuals_f0ee75e7-18ff-4260-ac5c-3db2ec0f8fd4'
 'sony_laptop_manuals_a02cf092-3538-4646-ab93-8cae84a07ad2']


In [13]:
df.head(25)

,id,question,ground_truth,unique_values
0,bose_headphones manuals_9917ef89-897c-6524-250...,Where can I know the complete Declaration of C...,You can search the website at www.Bose.com/com...,bose_headphones manuals_9917ef89-897c-6524-250...
1,bose_headphones manuals_9917ef89-897c-6524-250...,What does my Bose mobile on-ear headset include?,"It includes on-ear headphones, a headphone cor...",bose_headphones manuals_9917ef89-897c-6524-250...
2,bose_headphones manuals_9917ef89-897c-6524-250...,How many types of adapters are there for mobil...,"Each adapter has a symbol of a circle, triangl...",bose_headphones manuals_9917ef89-897c-6524-250...
3,bose_headphones manuals_9917ef89-897c-6524-250...,If my phone needs to use an adapter to connect...,You should find the appropriate adapter and co...,bose_headphones manuals_9917ef89-897c-6524-250...
4,bose_headphones manuals_9917ef89-897c-6524-250...,How can I put the headset in the case?,You can rotate the earcups until the cushions ...,bose_headphones manuals_9917ef89-897c-6524-250...
5,bose_headphones manuals_9917ef89-897c-6524-250...,How can I take the headset out of the case?,You can lift the headset out of the case and p...,bose_headphones manuals_9917ef89-897c-6524-250...
6,bose_headphones manuals_9917ef89-897c-6524-250...,How can I clean the headset?,You need to be sure the earcup ports are kept ...,bose_headphones manuals_9917ef89-897c-6524-250...
7,bose_headphones manuals_9917ef89-897c-6524-250...,What should I make sure of when reattaching ea...,You can make sure the cushion is flush all the...,bose_headphones manuals_9917ef89-897c-6524-250...
8,bose_headphones manuals_9917ef89-897c-6524-250...,What can I do if I still need help?,You can call for assistance if I still need help.,bose_headphones manuals_9917ef89-897c-6524-250...
9,bose_headphones manuals_9917ef89-897c-6524-250...,What is the official website of Bose in Japan?,The official website of Bose in Japan is www.B...,bose_headphones manuals_9917ef89-897c-6524-250...


In [14]:
df.drop(columns=['unique_values'], inplace=True)

/tmp/ipykernel_1503314/786108205.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['unique_values'], inplace=True)


In [15]:

questions = df['question'].tolist()
ground_truth = df['ground_truth'].tolist()
ground_truths = [[value] for value in ground_truth]

print(questions)
print(ground_truths)
answers = []
contexts = []

['Where can I know the complete Declaration of Conformity?', 'What does my Bose mobile on-ear headset include?', 'How many types of adapters are there for mobile phones?', 'If my phone needs to use an adapter to connect the headset. What should I do?', 'How can I put the headset in the case?', 'How can I take the headset out of the case?', 'How can I clean the headset?', 'What should I make sure of when reattaching earcup cushions?', 'What can I do if I still need help?', 'What is the official website of Bose in Japan?', 'Can I use headphones when operating a motor vehicle?', 'What does the Bose OE2 audio headphone include?', 'Can I exceed or rotate the earcups?', 'Where can I find replacement parts and accessories?', 'What should I do if any problems persists?', 'What are the steps to reattach earcup cushion?', 'How to obtain Limited Warranty Service?', 'What do you think of the hazardous standard of plastic parts?', 'What is the telephone number of the Bose corporation in Denmark?\n'

In [16]:
'''print(len(questions))


a1= []
a2= []
a3= []
a4= []
a5 = []
a6 = []
a7 = []
a8 = []
a9 = []
a10 = []

q1 = questions[0:10]
q2 = questions[10:20]
q3 = questions[20:30]
q4 = questions[30:40]
q5 = questions[40:50]
q6 = questions[50:60]
q7 = questions[60:70]
q8 = questions[70:80]
q9 = questions[80:90]
q10 = questions[90:100]
# q4 = questions[30:40]
# q5 = questions[40:50]'''


'print(len(questions))\n\n\na1= []\na2= []\na3= []\na4= []\na5 = []\na6 = []\na7 = []\na8 = []\na9 = []\na10 = []\n\nq1 = questions[0:10]\nq2 = questions[10:20]\nq3 = questions[20:30]\nq4 = questions[30:40]\nq5 = questions[40:50]\nq6 = questions[50:60]\nq7 = questions[60:70]\nq8 = questions[70:80]\nq9 = questions[80:90]\nq10 = questions[90:100]\n# q4 = questions[30:40]\n# q5 = questions[40:50]'

In [17]:
num_lists = len(questions) // 5 + (len(questions) % 5 > 0)  # Calculate the number of lists needed
splitted_questions = [questions[i * 5 : (i + 1) * 5] for i in range(num_lists)] 
empty_lists = [f'a{i} = []' for i in range(1, 21)]

In [18]:
splitted_questions[1]

['How can I take the headset out of the case?',
 'How can I clean the headset?',
 'What should I make sure of when reattaching earcup cushions?',
 'What can I do if I still need help?',
 'What is the official website of Bose in Japan?']

In [19]:
answers = []

In [20]:
for sublist_index, sublist in enumerate(splitted_questions, start=1):
    if sublist_index > 10:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers.append(answer)

Index: 0
Where can I know the complete Declaration of Conformity?
 To know the complete Declaration of Conformity for this device, you can refer to the safety information and instructions provided in the packaging or in the user manual that comes with the device. This information should include details about the device's compliance with specific regulations and standards, as well as any precautions or warnings you need to be aware of while using the device. It is important to follow these guidelines to ensure both your safety and the safe operation of the device. 
Index: 1
What does my Bose mobile on-ear headset include?
 Your Bose mobile on-ear headset includes on-ear headphones, a headphone cord with inline microphone, and four 3.5mm to 2.5mm mobile phone adapters. The headset is also compatible with many music-enabled mobile phones through the use of these adapters. To find out which adapter, if any, to use with your mobile phone, refer to the included compatibility chart. Without a

In [21]:
answers2 = []

In [22]:
for sublist_index, sublist in enumerate(splitted_questions[10:], start=11):
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers2.append(answer)

Index: 0
How many buttons can I find on the front of the device?


 Based on the provided context, there is one button on the front of the device: the Talk button. 
Index: 1
Are the items supplied with the devices and available accessories the same in different regions?
 The items supplied with the devices and available accessories may not be exactly the same in different regions. Manufacturers like Samsung may have regional variants or localized versions of their products, which can include differences in the packaging, user manuals, cables, chargers, and other accessories. Additionally, some devices may require a SIM card for activation, and service carriers' coverage areas may vary by region. It is essential to check the compatibility of your device with local network providers before attempting to use it in another country or region. 
Index: 2
When should i charge the battery completely?
 You should charge the battery completely after you have set up your phone for the first time, as it is an important step to ensure that the device functions prop

In [23]:
print(len(answers))
print(len(answers2))
print(len(questions))

50
50
100


In [24]:
print(duck)

NameError: name 'duck' is not defined

In [ ]:
print(len(answers))

0


In [ ]:
print(len(splitted_questions))

20


In [ ]:
for sublist_index, sublist in enumerate(splitted_questions, start=1):
    if sublist_index > 3:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers.append(answer)

Index: 0
Where can I know the complete Declaration of Conformity?
 Sound,” “Stereo Wide,” and so on. While these features can enhance your listening experience with certain types of music, they are generally not designed to be used with headphones like the Bose mobile on-ear headset. To get the best sound quality from your Bose mobile on-ear headset, set your audio device to “normal,” or “flat” sound setting and listen.
Index: 1
What does my Bose mobile on-ear headset include?
 This appears to be a user manual for a Bose SoundLink audio system designed for use with computers. The manual covers topics such as product specifications, warranty information, and instructions for connecting and using the system with a computer. Additionally, it includes troubleshooting steps for common issues that may arise during setup or usage. 
Index: 2
How many types of adapters are there for mobile phones?
 Biphenyl) Dioxide (PBD) 
Index: 3
If my phone needs to use an adapter to connect the headset. Wha

In [ ]:
print(len(answers))

15


In [ ]:
answers

[' Sound,” “Stereo Wide,” and so on. While these features can enhance your listening experience with certain types of music, they are generally not designed to be used with headphones like the Bose mobile on-ear headset. To get the best sound quality from your Bose mobile on-ear headset, set your audio device to “normal,” or “flat” sound setting and listen.',
 ' This appears to be a user manual for a Bose SoundLink audio system designed for use with computers. The manual covers topics such as product specifications, warranty information, and instructions for connecting and using the system with a computer. Additionally, it includes troubleshooting steps for common issues that may arise during setup or usage. ',
 ' Biphenyl) Dioxide (PBD) ',
 " The image you've provided is a product manual for the Bose OE2 audio headphones. This guide contains information about the proper use, cleaning, and troubleshooting of the headphones, as well as details on warranty and return policies. It also in

In [ ]:
answers2 = []

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[3:], start=4):
    if sublist_index > 5:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers2.append(answer)

Index: 0
What are the steps to reattach earcup cushion?
 This is an instruction manual for a Bluetooth headset. The headset can be used with mobile phones and other audio devices with 3.5mm headphone jacks. It features noise reduction technology, and has a battery life of up to 20 hours on standby and up to 6 hours of talk time. The charging time is approximately 2 hours. The warranty is valid for one year from the date of purchase, with proof of purchase required for service. The product registration card should be completed and returned to Bose within a certain period for warranty coverage. 
Index: 1
How to obtain Limited Warranty Service?
 
Index: 2
What do you think of the hazardous standard of plastic parts?
 Front Panel has the following parts:
1. Power Button:这个按钮用于控制器的能力。
2.通道:这是用户手指的方向来控制器。
3.前进区:这种区域可以用于控制器的功能。
4.后进区:这种区域可以用于控制器的特定功能。
5.前进进行的区域:这些区域可以用于控制器的特别功能。
6.后进进行的区域:这种区域可以用于控制器的特别功能。
7.前进进行的区域:这个区域可以用于控制器的特别功能。
8.后进进行的区域:这个区域可以用于控制器的特别功能。
9.前进进行的区域:这种区域可以用于控制器的特别功能。
10.

In [ ]:
print(len(answers2))

10


In [ ]:
answers3 = []

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[5:], start=6):
    if sublist_index > 7:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers3.append(answer)

Index: 0
How can I turn on the phone?
 21

+ Battery pack (optional) — Provides additional battery life for your smartphone.

+ Carrying case — Protects your smartphone from damage caused by accidental drops or mishandling.

[& NOTE: For more information on the accessories that ship with your smartphone, see the User's Manual on the Dell Support website at supportapj.dell.com.

For additional details and specifications about your Mini 3i Smartphone, refer to the following resources:

+ Dell website — Visit Dell.com for product-specific
information, software updates, and other online resources.

* Dell Support community — Access the Dell Support community
at supportapj.dell.com to find answers from other users
or ask questions of your own.

* User's Manual — Review the printed manual that came
with your smartphone for detailed instructions and troubleshooting tips.

* Product Support Center — Contact Dell
Product Support by phone or email to get help with
any technical issue you may enc

In [ ]:
print(len(answers3))

10


In [ ]:
answers2

[' This is an instruction manual for a Bluetooth headset. The headset can be used with mobile phones and other audio devices with 3.5mm headphone jacks. It features noise reduction technology, and has a battery life of up to 20 hours on standby and up to 6 hours of talk time. The charging time is approximately 2 hours. The warranty is valid for one year from the date of purchase, with proof of purchase required for service. The product registration card should be completed and returned to Bose within a certain period for warranty coverage. ',
 ' ',
 ' Front Panel has the following parts:\n1. Power Button:这个按钮用于控制器的能力。\n2.通道:这是用户手指的方向来控制器。\n3.前进区:这种区域可以用于控制器的功能。\n4.后进区:这种区域可以用于控制器的特定功能。\n5.前进进行的区域:这些区域可以用于控制器的特别功能。\n6.后进进行的区域:这种区域可以用于控制器的特别功能。\n7.前进进行的区域:这个区域可以用于控制器的特别功能。\n8.后进进行的区域:这个区域可以用于控制器的特别功能。\n9.前进进行的区域:这种区域可以用于控制器的特别功能。\n10.后进进行的区域:这个区域可以用于控制器的特别功能。\n11.后进进行的区域:这种区域可以用于控制器的特别功能。\n12.后进进行的区域:这种区域可以用于控制器的特别功能。\n13.后进进行了区域：这个区域可以用于控制器的特别功能。\n14.后进进行的区域：这个区域可以用于控制器的特别功能。\n15.后进进行了区

In [ ]:
answers3

[" 21\n\n+ Battery pack (optional) — Provides additional battery life for your smartphone.\n\n+ Carrying case — Protects your smartphone from damage caused by accidental drops or mishandling.\n\n[& NOTE: For more information on the accessories that ship with your smartphone, see the User's Manual on the Dell Support website at supportapj.dell.com.\n\nFor additional details and specifications about your Mini 3i Smartphone, refer to the following resources:\n\n+ Dell website — Visit Dell.com for product-specific\ninformation, software updates, and other online resources.\n\n* Dell Support community — Access the Dell Support community\nat supportapj.dell.com to find answers from other users\nor ask questions of your own.\n\n* User's Manual — Review the printed manual that came\nwith your smartphone for detailed instructions and troubleshooting tips.\n\n* Product Support Center — Contact Dell\nProduct Support by phone or email to get help with\nany technical issue you may encounter. ",
 ' 

In [ ]:
answers4 = []

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[6:], start=7):
    if sublist_index > 8:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers4.append(answer)

Index: 0
I want to use this product outdoors. Is that OK?


 This is a user manual for the Bose SoundLink USB key and adapter. The document provides information about the product's features, setup, and safety precautions. It also includes warranty and disposal instructions. The manual is intended for users in North America and contains additional safety information on the Important Safety Instructions sheet enclosed in the shipping carton. 
Index: 1
How can I know whether this equipment causes interference to television reception or not?
 Sound.” Please be aware that using these additional features can impact the performance and audio quality of your Bose mobile on-ear headset. If you are unsure about the compatibility of any audio device with the Bose mobile on-ear headset, or if you need further assistance, please contact our customer service department at 1 (800) 379-4650.

Thank you for choosing the Bose” mobile on-ear headset. We hope that it provides you with many hours of enjoyment. If you have any questions or concerns about your produc

In [ ]:
print(len(answers4))

10


In [ ]:
answers4

[" This is a user manual for the Bose SoundLink USB key and adapter. The document provides information about the product's features, setup, and safety precautions. It also includes warranty and disposal instructions. The manual is intended for users in North America and contains additional safety information on the Important Safety Instructions sheet enclosed in the shipping carton. ",
 ' Sound.” Please be aware that using these additional features can impact the performance and audio quality of your Bose mobile on-ear headset. If you are unsure about the compatibility of any audio device with the Bose mobile on-ear headset, or if you need further assistance, please contact our customer service department at 1 (800) 379-4650.\n\nThank you for choosing the Bose” mobile on-ear headset. We hope that it provides you with many hours of enjoyment. If you have any questions or concerns about your product, please do not hesitate to contact our customer service department at 1 (800) 379-4650.\n

In [ ]:
answers5 = []

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[8:], start=9):
    if sublist_index > 10:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers5.append(answer)

Index: 0
What does the exclamation point within an equilateral triangle mean?
 Bisphenol A) Flame Retardants | Dioxins & PCBs
Electrical Components | Formaldehyde
Plastic Parts | Lead
Headband | Mercury | Cadmium Hexavalent
Cushion Foam | Polybrominated Biphenyl
Ear Cup | Cadmium Hexavalent | Flame Retardants
Speaker | Lead | Polybrominated Biphenyl | Dioxins & PCBs
Auxiliary Input Jack | Mercury | Cadmium Hexavalent | Flame Retardants
Charging Connector | Formaldehyde | Flame Retardants | Dioxins & PCBs
 
Names and Contents of Toxic or Hazardous Substances or Elements
Part | Lead] Mercury | Cadmium Hexavalent | Polybrominated Biphenol| Flame Retardants| Dioxins & PCBs| Lead|Cadmium Hexavalent|Polybrominated Biphenol|Flame retardants|Dioxins & PCBs
Names | _Polybrominated Biphenol| _Formalehyde| _Polybrominated Biphenol| Dioxins & PCBs| _Lead_| _Mercury_| _Cadmium Hexavalent_| _Polybrominated Biphenol_| _Flame retardants_ | _Dioxins & PCBs_| _Lead_| _Mercury_| _Mercury_| _Cadmium Hexav

In [ ]:
print(len(answers5))

10


In [ ]:
answers5

[' Bisphenol A) Flame Retardants | Dioxins & PCBs\nElectrical Components | Formaldehyde\nPlastic Parts | Lead\nHeadband | Mercury | Cadmium Hexavalent\nCushion Foam | Polybrominated Biphenyl\nEar Cup | Cadmium Hexavalent | Flame Retardants\nSpeaker | Lead | Polybrominated Biphenyl | Dioxins & PCBs\nAuxiliary Input Jack | Mercury | Cadmium Hexavalent | Flame Retardants\nCharging Connector | Formaldehyde | Flame Retardants | Dioxins & PCBs\n \nNames and Contents of Toxic or Hazardous Substances or Elements\nPart | Lead] Mercury | Cadmium Hexavalent | Polybrominated Biphenol| Flame Retardants| Dioxins & PCBs| Lead|Cadmium Hexavalent|Polybrominated Biphenol|Flame retardants|Dioxins & PCBs\nNames | _Polybrominated Biphenol| _Formalehyde| _Polybrominated Biphenol| Dioxins & PCBs| _Lead_| _Mercury_| _Cadmium Hexavalent_| _Polybrominated Biphenol_| _Flame retardants_ | _Dioxins & PCBs_| _Lead_| _Mercury_| _Mercury_| _Cadmium Hexavalent_| _Polybrominated Biphenol_| _Flame retardants_ | _Polybro

In [ ]:
answers6 = []

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[10:], start=11):
    if sublist_index > 12:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers6.append(answer)

Index: 0
How many buttons can I find on the front of the device?
 
Index: 1
Are the items supplied with the devices and available accessories the same in different regions?
 The text in the image is too small to read clearly. 
Index: 2
When should i charge the battery completely?
 Support on Bose.com for a list of Bose Customer Service contacts around the world) to obtain a Return Merchandise Authorization (RMA) number and instructions for returning your product. If you need assistance, please contact the Bose Customer Service organization in your country/region.

1. Send the product, along with the RMA number and any other required documentation, to the address specified by Bose Customer Service. Shipping is at your expense. Please do not send the product to any other location without authorization.

1. Keep your proof of purchase handy in case it is needed for warranty service.

This limited warranty gives you specific legal rights, and you may also have other rights which vary from 

In [ ]:
print(len(answers6))

10


In [ ]:
answers6

[' ',
 ' The text in the image is too small to read clearly. ',
 ' Support on Bose.com for a list of Bose Customer Service contacts around the world) to obtain a Return Merchandise Authorization (RMA) number and instructions for returning your product. If you need assistance, please contact the Bose Customer Service organization in your country/region.\n\n1. Send the product, along with the RMA number and any other required documentation, to the address specified by Bose Customer Service. Shipping is at your expense. Please do not send the product to any other location without authorization.\n\n1. Keep your proof of purchase handy in case it is needed for warranty service.\n\nThis limited warranty gives you specific legal rights, and you may also have other rights which vary from country. ',
 ' Brominated Flame Retardants)\nBrominated Fluorocarbons\n\nNotes:\n\n1. This product contains one or more of the chemicals listed above in a concentration that is within the allowable limits unde

In [ ]:
answers7 = []

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[12:], start=13):
    if sublist_index > 14:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers7.append(answer)

Index: 0
What is the World Wide Web of the SAMSUNG?
 Using Your Mini 3i Smartphone

Connecting a Bluetooth Headset

1. Turn on your Bluetooth headset.

2. On your smartphone, go to Settings > Bluetooth.

 

3. If Bluetooth is not already enabled, tap the switch next to "Bluetooth" to turn it on.

4. Once your headset appears in the list of available devices on the screen, tap the name of your Bluetooth headset to connect it to your smartphone.

 

5. If prompted, enter the passkey displayed on your smartphone's screen on your headset. Thepasskey is a
Index: 1
How to avoid damaging the performance and reducing the battery life?
 Sound” or “virtual surround sound.” When using your Bose® mobile on-ear headset with these devices, you can choose to activate these features to enhance your listening experience. However, keep in mind that some of these features may alter the natural audio balance and tonal quality provided by the Bose® mobile on-ear headset.

UsiNG Your HEADSET

Using the head

In [ ]:
print(len(answers7))

10


In [ ]:
answers7

[' Using Your Mini 3i Smartphone\n\nConnecting a Bluetooth Headset\n\n1. Turn on your Bluetooth headset.\n\n2. On your smartphone, go to Settings > Bluetooth.\n\n \n\n3. If Bluetooth is not already enabled, tap the switch next to "Bluetooth" to turn it on.\n\n4. Once your headset appears in the list of available devices on the screen, tap the name of your Bluetooth headset to connect it to your smartphone.\n\n \n\n5. If prompted, enter the passkey displayed on your smartphone\'s screen on your headset. Thepasskey is a',
 ' Sound” or “virtual surround sound.” When using your Bose® mobile on-ear headset with these devices, you can choose to activate these features to enhance your listening experience. However, keep in mind that some of these features may alter the natural audio balance and tonal quality provided by the Bose® mobile on-ear headset.\n\nUsiNG Your HEADSET\n\nUsing the headphone cord with inline microphone\nTo answer a call while using your Bose® mobile on-ear headset, simpl

In [ ]:
answers8 = []

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[13:], start=14):
    if sublist_index > 15:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers8.append(answer)

Index: 0
What is the battery level if the indicator light flashes blue?
 The text provides information about the product and its usage. It includes details such as the warranty, specifications, and instructions for pairing and connecting the headset to a Bluetooth device. There are also answers to some questions that may arise during the use of this product. 
Index: 1
What will happen if my headset powers off during a call?
 .Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;

2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and

3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused.

Note: The serial number is located on the warranty card included in the carton.

16

Index: 2
How can I know if I turn off my headset?
 Bose.com/register 

In [ ]:
print(len(answers8))

10


In [ ]:
answers8

[' The text provides information about the product and its usage. It includes details such as the warranty, specifications, and instructions for pairing and connecting the headset to a Bluetooth device. There are also answers to some questions that may arise during the use of this product. ',
 ' .Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused.\n\nNote: The serial number is located on the warranty card included in the carton.\n\n16\n',
 ' Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address pro

In [ ]:
answers9 = []

In [ ]:
saved = [' Sound,” “Stereo Wide,” and so on. While these features can enhance your listening experience with certain types of music, they are generally not designed to be used with headphones like the Bose mobile on-ear headset. To get the best sound quality from your Bose mobile on-ear headset, set your audio device to “normal,” or “flat” sound setting and listen.',
 ' This appears to be a user manual for a Bose SoundLink audio system designed for use with computers. The manual covers topics such as product specifications, warranty information, and instructions for connecting and using the system with a computer. Additionally, it includes troubleshooting steps for common issues that may arise during setup or usage. ',
 ' Biphenyl) Dioxide (PBD) ',
 " The image you've provided is a product manual for the Bose OE2 audio headphones. This guide contains information about the proper use, cleaning, and troubleshooting of the headphones, as well as details on warranty and return policies. It also includes a diagram of how to fold and store the headset when not in use.\n\nThe headset appears to be designed for portable use, with an adjustable headband and 3.5mm audio cable that can be connected to various devices. The manual emphasizes the importance of not using the headphones as a walking device or connecting them to unsecured devices. Additionally, it advises users to avoid using any liquids to clean the device and to refer to the warranty card for warranty information. ",
 " It seems you've provided an image of product instructions for a vacuum cleaner. The text in the image includes information about the warranty, cleaning tools, cord storage, maintenance steps such as emptying the dust bin and cleaning the grille or pre-motor filter, and replacing the post-motor filter. If you have any questions related to this image of product instructions, feel free to ask. ",
 " The image you have shared appears to show an instruction manual for a Bose product, specifically for their wireless headset. Here's a summary of the information provided in the image:\n\n1. How do I charge my headset?\n* Connect the headset to the charger using the cable.\n* Place the headset and travel adapter on a flat surface or attach the travel adapter to your wall outlet.\n1. What is the battery life of the headset?\n* The headset has a talk time of up to 6 hours, but this may vary depending on the phone type and usage.\n1. Limited Warranty:\n* Samsung warrants the product for one year from the date of purchase.\n* The warranty is limited to the original purchaser and voids if subjected to physical abuse or improper installation by unauthorized third parties.\n* Proof of purchase (e.g., receipt) is required for warranty service.\n1. Safety Instructions:\n* Follow proper safety guidelines when using the product.\n* Use an attachment plug adaptor for a supply not in North America, with the correct configuration for the power outlet.\n1. Warranty and parts replacement:\n* The warranty is limited to one year from the date of purchase.\n* It is void if subjected to physical abuse or improper installation by unauthorized third parties.\n* Proof of purchase is required for warranty service. ",
 ' Operating the cleaner ',
 ' The image shows a mobile on-ear headset with a focus on its charging features and instructions for use. It highlights that the headset can be used wirelessly through Bluetooth connectivity, or wired with the included 3.5mm audio cable. The packaging also indicates that it is compatible with various devices, and includes a travel adapter for international use. Additionally, the image emphasizes safety precautions and warranty information for the product. ',
 ' This image appears to be a page from the user manual for the Bose SoundLink Audio Headset. The page contains safety instructions, warranty information, and troubleshooting steps for issues related to computer connectivity, sound playback, and linking the headset with a computer. The text on the page is too small to read in detail, but it provides guidance on how to resolve common problems that users may encounter when using the headset with a computer. ',
 ' ',
 ' This appears to be a user manual for a wireless audio system. It includes information on how to use the product, troubleshooting common issues, warranty information, and contact information for customer service. The document is in English and has images of the product, its components, and some of its features. ',
 ' The Bose® OE2 audio headphones do not contain any hazardous materials.\n\n ',
 ' Phone: +64 9636537\nFax: +64 9636538\nwww.Bose.co.nz\n\nNorway\nPhone: 0230-1130\nFax:  0230-1131\nwww.Bose.no\n\nPhilippines\nPhone: (02) 764-7859\nFax: (02) 764-7856\nwww.Bose.ph\n\nPoland\n\nPhone: +48 22 310-500\nFax: +48 22 310-501\nwww.Bose.com.pl\n\nPortugal\n\nPhone: 0291 967 785 3753\nFax: 0291 967 785 3754\nwww.Bose.pt\n\nRussia\nPhone: +7 (0) 2 904 604 10 00\nFax: +7 (0) 2 904 604 10 01\nwww.Bose.com.ru\n\nSingapore\nPhone: (06) 535-8800\nFax: (06) 535-8807\nwww.Bose.com.sg\n\nSpain\n\nPhone: +34 91 203 8346\nFax: +34 91 203 8347\nwww.Bose.es\n\nSweden\n\nPhone: 020-15771\nFax:  020-157719\nwww.Bose.se\n\nSwitzerland\n\nPhone: +41 (0) 21 663 83 61\nFax: +41 (0) 21 663 83 60\nwww.Bose.ch\n\nTaiwan\n\nPhone: 02-859-7608\nFax:  02-859-7607\nwww.Bose.com.tw\n\nThailand\nPhone: (0) 211-306-785-3853\nFax: (0) 211-306-785-3854\nwww.Bose.co.th\n\nUnited Kingdom\n\nPhone: 0208 882 200\nFax:  0208 882 2001\nwww.Bose.co.uk\n\nUnited States\n\nPhone: 1-800-465-2673\nFax: 1-800-862-2673\nwww.Bose.com\n\nUruguay\n\nPhone: +59 218-202 7420\nFax: +59 218-202 7419\nwww.Bose.com.uy\n\nVenezuela\n\nPhone: +58 (0) 213 100 2673\nFax: +58 (0) 213 100 2674\n\nPlease see the inside back cover for contact information in other countries. ',
 ' The Bluetooth word mark and logos are registered trademarks owned by Bluetooth SIG, Inc. and any use of such marks by Bose Corporation is under license. ',
 ' .Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused.\n\nNote: The serial number is located on the warranty card included in the carton.\n\n ', ' This is an instruction manual for a Bluetooth headset. The headset can be used with mobile phones and other audio devices with 3.5mm headphone jacks. It features noise reduction technology, and has a battery life of up to 20 hours on standby and up to 6 hours of talk time. The charging time is approximately 2 hours. The warranty is valid for one year from the date of purchase, with proof of purchase required for service. The product registration card should be completed and returned to Bose within a certain period for warranty coverage. ',
 ' ',
 ' Front Panel has the following parts:\n1. Power Button:这个按钮用于控制器的能力。\n2.通道:这是用户手指的方向来控制器。\n3.前进区:这种区域可以用于控制器的功能。\n4.后进区:这种区域可以用于控制器的特定功能。\n5.前进进行的区域:这些区域可以用于控制器的特别功能。\n6.后进进行的区域:这种区域可以用于控制器的特别功能。\n7.前进进行的区域:这个区域可以用于控制器的特别功能。\n8.后进进行的区域:这个区域可以用于控制器的特别功能。\n9.前进进行的区域:这种区域可以用于控制器的特别功能。\n10.后进进行的区域:这个区域可以用于控制器的特别功能。\n11.后进进行的区域:这种区域可以用于控制器的特别功能。\n12.后进进行的区域:这种区域可以用于控制器的特别功能。\n13.后进进行了区域：这个区域可以用于控制器的特别功能。\n14.后进进行的区域：这个区域可以用于控制器的特别功能。\n15.后进进行了区域：这个区域可以用于控制器的特别功能。\n16.后进进行的区域：这个区域可以用于控制器的特别功能。\n17.后进进行了区域:这个区域可以用于控制器的特别功能。\n18.后进进行了区域：这个区域可以用于控制器的特别功能。\n19.后进进行了区域：这个区域可以用于控制器的特别功能。\n20.前进进行的区域：这个区域可以用于控制器的特别功能。\n21.前进进行了区域：这个区域可以用于控制器的特别功能。\n22.前进进行了区域：这个区域可以用于控制器的特别功能。\n23.前进进行了区域：这个区域可以用于控制器的特别功能。\n24.前进进行了区域:这个区域可以用于控制器的特别功能。\n25.前进进行了区域:这个区域可以用于控制器的特别功能。\n26.前进进行了区域：这个区域可以用于控制器的特别功能。\n27.前进进行了区域:这个区域可以用\n12.后进进行了区域:这个区域可以用于控制器的特别功能。\n28.后进进行了区域：这个区域可以用于控制器的特别功能。\n29.后进进行了区域:这个区域可以用于控制器的特别功能。\n30.后进进行了区域:这个区域可以用于控制器的特别功能。\n31.后进进行了区域:这个区域可以用',
 ' 11\n\nTroubleshooting\n\nFor more help with your Wave® music system, visit the Bose Customer Service Help System at www.bose.com or contact us by phone at 800-572-432. Please have your serial number available when you call.\n\nIf you encounter any problems or issues not covered in this user guide, please refer to the Bose Customer Service Help System or contact our customer service representatives for further assistance.\n\nPlease note that warranty and support coverage may vary by region or country. If your Wave® music system is not operating properly and you believe it may be a defective product, contact Bose Customer Service to learn about the warranty and service options available in your location.\n\nWave® Music System\n575 Broadway, Suite 100\nCambridge, MA 02139 USA\nPhone: +1 (617) 489-3651\n\nFax: +1 (617) 489-8189\nEmail: [service@bose.com](mailto:service@bose.com)\n\nOnline Support: www.bose.com/support\nLive Chat: http://www.bose.com/livechat\n\nPlease note that some features may not be available in all countries or regions. The Wave® music system is compatible with Windows® 2000, Windows XP, Windows Vista, and Windows 7 operating systems. It is not compatible with Macintosh operating systems. ',
 ' Biphenyl | Bisphenol A (BPA) | Chlorine | Chloroform | Chlorobenzene | Chlorofluorocarbons |\nChloromethane | Chlorophenols | Chlorotrifluoroethane | Chlorotoluene | Chloroxylene |\nChlorpyridines | Chlorpyrene | Chlorpyridine | Chlorpyrene | Chloroform | Chlorothiophene |\nChloropropionates | Chloropyridones | Chlorpropofol | Chlorosulfonic acid |\n1,2-Dichlorobenzene | 1,3-Dichlorobenzene | Dichlorodiphenyltrichloroethane (DDT) |\n1,2,3-Trichloropropyl chloride | Chlorotoluene | Chloroxylene |\nChlorpyrene | Chloropyrene | Chlorpyrene | Chlorothiophene |\n\n ',
 ' This is a remote controler specifically designed to operate and manage A series cameras through At coaxial connection, and COAXITRON supporting cameras through COAXITRON communication. It features easy-to-use controls with a total of 8 keys: +, -, Up, Down, Left, Right, Home, Power, Menu,P\xa0',
 ' Bisphenol A) ',
 ' Diphenyl ether) Chlorine Tetrabromo[1,3,5-trimethyl benzene] Bromine | Pentafluoride\nBromine Heptabromo[1,3,5-trimethyl benzene] Bromine | Pentafluoride\nChlorine Tris(trifluoromethyl) phosphate | Polybrominated diphenyl ether\nDichlorodiphenyl trichloroethane (DDT) | Tetrabromo[1,3,5-trimethyl benzene]\nDibromodichloropropane (DBDP) | Pentafluoride\nDi(2-propyl)phthalate | Chlorine tris(trifluoromethyl) phosphate\nPentachlorophenol | Tetrabromo[1,3,5-trimethyl benzene]\nPentabromodiphenyl ether | Pentafluoride\nPolychlorinated biphenyls (PCBs) | Chlorine tris(trifluoromethyl) phosphate\nPolybrominated diphenyl ether | Dichlorodiphenyl trichloroethane (DDT)\nTrichlorodiphenyl trichloroethane (TDDE) | Tetrabromo[1,3,5-trimethyl benzene]\nTris(chloropropyl) phosphate | Dibromodichloropropane (DBDP)\nTris(2-propyl)phthalate | Pentachlorophenol\nTetrabromobenzene | Pentafluoride\nTetrabromoethylbenzene | Chlorine tris(trifluoromethyl) phosphate\nTetrachlorodibutyl ether (TCBDE) | Polybrominated diphenyl ether\nTrichlorodiphenyl dichloroethane (TDDE) | Pentafluoride\nTris(chloropropyl)phosphate | Dichlorodiphenyl trichloroethane (DDT) ',
 ' Sound” or other virtual surround sound options. It is important to review the documentation of your specific device and follow its instructions to ensure optimal performance with your Bose mobile on-ear headset. Please note that some audio devices may not be compatible with your Bose mobile on-ear headset, such as certain gaming consoles or older generation iPods.\n\n ',
 ' Address Book](http://www.bose.com/en_us/support/global_address_book.html) for contact information.\n2. The return shipping cost is not covered under our warranty.\n3. Allow 10 to 15 business days for the repair or replacement to be processed.\n4. Bose may charge a restocking fee of up to 25% of the purchase price, based on the severity and nature of the damage.\n\nThis limited warranty covers parts that fail under normal use during the 1-year period from the date of original retail purchase. It does not cover damages caused by accident, abuse, misuse, or failure to follow Bose product instructions.\n\nCustomer Service Address Book](http://www.bose.com/en_us/support/global_address_book.html) ' , " 21\n\n+ Battery pack (optional) — Provides additional battery life for your smartphone.\n\n+ Carrying case — Protects your smartphone from damage caused by accidental drops or mishandling.\n\n[& NOTE: For more information on the accessories that ship with your smartphone, see the User's Manual on the Dell Support website at supportapj.dell.com.\n\nFor additional details and specifications about your Mini 3i Smartphone, refer to the following resources:\n\n+ Dell website — Visit Dell.com for product-specific\ninformation, software updates, and other online resources.\n\n* Dell Support community — Access the Dell Support community\nat supportapj.dell.com to find answers from other users\nor ask questions of your own.\n\n* User's Manual — Review the printed manual that came\nwith your smartphone for detailed instructions and troubleshooting tips.\n\n* Product Support Center — Contact Dell\nProduct Support by phone or email to get help with\nany technical issue you may encounter. ",
 ' ',
 ' New message\nForward message\nCompose new message\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n17\n\nSending and Receiving Text Messages (SMS) or Multimedia Messages (MMS)\n\n1. Tap the Messaging icon ©\n\n2. Press the Options button to display a short menu on the bottom of the screen\n\n \n\n3. Using the short menu, you can compose\n [/\n\nNew message\nForward message\nCompose new message\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n18\n\nUsing Your Mini 3i Smartphone\n\nAdding Contacts\n\n1. Tap the Contacts icon ©\n\n2. Press the Options button to display a short menu on the bottom of the screen\n\n \n\n3. Using the short menu, you can add a new contact\n [/\n\nAdd contact\nImport contacts\nExport contacts\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n19\n\nUsing Your Mini 3i Smartphone\n\nSetting Up Email Accounts\n\n1. Tap the Email icon ©\n\n2. Press the Options button to display a short menu on the bottom of the screen\n\n \n\n3. Using the short menu, you can set up an email account\n [/\n\nAdd email account\nImport email account\nExport email account\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n20\n\nUsing Your Mini 3i Smartphone\n\nConnecting to the Internet\n\n1. Press and hold the Power button ©\n\n2. When prompted, select Wi-Fi from the menu\n\n \n\n3. Select your desired network from the list of available networks.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n21\n\nUsing Your Mini 3i Smartphone\n\nTurning Your Phone On or Off\n\n1. Press and hold the Power button ©\n\n2. When prompted, select "Power off" from the menu\n\n \n\n3. The screen will turn black to indicate that the phone is turned off.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n22\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device\n\n1. Connect your phone to a computer using a USB cable.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n23\n\nUsing Your Mini 3i Smartphone\n\nConnecting to Bluetooth Devices\n\n1. Press and hold the Power button ©\n\n2. When prompted, select "Bluetooth" from the menu\n\n \n\n3. Turn on your Bluetooth device and make sure that it is paired with your phone.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n24\n\nUsing Your Mini 3i Smartphone\n\nAdjusting Audio Settings\n\n1. Press the Volume button to adjust the volume or\nmute the microphone.\n\n2. Press and hold the Power button ©\n\n3. When prompted, select "Audio settings" from the menu\n\n \n\n4. Adjust the audio settings as needed\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n25\n\nUsing Your Mini 3i Smartphone\n\nSetting Up Alarms or Calendar Events\n\n1. Tap the Clock icon ©\n\n2. Press the Options button to display a short menu on the bottom of the screen\n\n \n\n3. Using the short menu, you can set up alarms or calendar events\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n26\n\nUsing Your Mini 3i Smartphone\n\nListening to Music, Videos, or Podcasts\n\n1. Connect your phone to a computer using a USB cable.\n\n2. Transfer music, videos, or podcasts onto your phone\'s memory card or internal storage.\n\n3. Play the desired media file from your phone\'s media player application.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n27\n\nUsing Your Mini 3i Smartphone\n\nTaking Photos or Videos\n\n1. Press the Camera button to open the camera application.\n\n2. Take photos or record videos using your phone\'s built-in camera.\n\n3. Save the media file onto your phone\'s memory card or internal storage for later use or sharing with others.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n28\n\nUsing Your Mini 3i Smartphone\n\nCharging Your Phone\n\n1. Connect your phone to a computer using a USB cable.\n\n2. Allow your phone to charge until it reaches full battery capacity.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n29\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Television or Other Display Device\n\n1. Connect your phone to a television or other display device using an HDMI cable.\n\n2. Select the desired media file from your phone\'s gallery application.\n\n3. The media file will be displayed on the connected television or other display device.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n30\n\nUsing Your Mini 3i Smartphone\n\nConnecting to Other Bluetooth Devices\n\n1. Connect your phone to a Bluetooth headset or speaker using an audio cable with a built-in microphone.\n\n2. Answer incoming calls, adjust the volume or mute the microphone using the phone\'s controls.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n31\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Laptop or Other Computer\n\n1. Connect your phone to a laptop or other computer using a USB cable.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n32\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Wired Headset or Earbuds\n\n1. Connect your phone to a wired headset or earbuds using an audio cable with a built-in microphone.\n\n2. Answer incoming calls, adjust the volume, or mute the microphone using the phone\'s controls.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n33\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via Wi-Fi\n\n1. Connect your phone to a computer or other device using a Wi-Fi connection.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n34\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Bluetooth Headset or Speaker via Wi-Fi\n\n1. Connect your phone to a Bluetooth headset or speaker using a Wi-Fi connection.\n\n2. Answer incoming calls, adjust the volume, or mute the microphone using the phone\'s controls.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n35\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via Bluetooth\n\n1. Connect your phone to a computer or other device using a Bluetooth connection.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n36\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Bluetooth Headset or Speaker via Bluetooth\n\n1. Connect your phone to a Bluetooth headset or speaker using a Bluetooth connection.\n\n2. Answer incoming calls, adjust the volume, or mute the microphone using the phone\'s controls.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n37\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via USB\n\n1. Connect your phone to a computer or other device using a USB connection.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n38\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Wired Headset or Earbuds via Bluetooth\n\n1. Connect your phone to a wired headset or earbuds using an audio cable with a built-in microphone.\n\n2. Answer incoming calls, adjust the volume, or mute the microphone using the phone\'s controls.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n39\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Wired Headset or Earbuds via USB\n\n1. Connect your phone to a wired headset or earbuds using an audio cable with a built-in microphone.\n\n2. Answer incoming calls, adjust the volume, or mute the microphone using the phone\'s controls.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n40\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Bluetooth Headset or Speaker via USB\n\n1. Connect your phone to a Bluetooth headset or speaker using an audio cable with a built-in microphone.\n\n2. Answer incoming calls, adjust the volume, or mute the microphone using the phone\'s controls.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n41\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via USB with Charging\n\n1. Connect your phone to a computer or other device using a USB connection with charging capabilities.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n3. Allow your phone to charge until it reaches full battery capacity.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n42\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via USB with File Transfer\n\n1. Connect your phone to a computer or other device using a USB connection with file transfer capabilities.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n43\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via Wi-Fi with File Transfer\n\n1. Connect your phone to a computer or other device using a Wi-Fi connection with file transfer capabilities.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n44\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via Bluetooth with File Transfer\n\n1. Connect your phone to a computer or other device using a Bluetooth connection with file transfer capabilities.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n45\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via USB with Charging and File Transfer\n\n1. Connect your phone to a computer or other device using a USB connection with charging and file transfer capabilities.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n3. Allow your phone to charge until it reaches full battery capacity.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n46\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via USB with Charging and Data Transfer\n\n1. Connect your phone to a computer or other device using a USB connection with charging and data transfer capabilities.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n3. Allow your phone to charge until it reaches full battery capacity.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n47\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via USB with File Transfer and Charging\n\n1. Connect your phone to a computer or other device using a USB connection with file transfer and charging capabilities.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n3. Allow your phone to charge until it reaches full battery capacity.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n48\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via USB with Data Transfer and Charging\n\n1. Connect your phone to a computer or other device using a USB connection with data transfer and charging capabilities.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n3. Allow your phone to charge until it reaches full battery capacity.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n49\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via Wi-Fi with Data Transfer and Charging\n\n1. Connect your phone to a computer or other device using a Wi-Fi connection with data transfer and charging capabilities.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n3. Allow your phone to charge until it reaches full battery capacity.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com.\n\n50\n\nUsing Your Mini 3i Smartphone\n\nConnecting to a Computer or Other Device via Bluetooth with Data Transfer and Charging\n\n1. Connect your phone to a computer or other device using a Bluetooth connection with data transfer and charging capabilities.\n\n2. Transfer files between the two devices, such as photos, videos, or music files.\n\n3. Allow your phone to charge until it reaches full battery capacity.\n\n[& NOTE: For more information on other options for this task, see the User\'s Manual on the Dell Support website at supportapj.dell.com. ',
 ' ',
 ' Customer Support at Owners.Bose.com for a list of authorized dealers and the address of the Bose office nearest you).\n2. Call the Bose toll-free number for your country/region, as listed on the warranty card or the product packaging.\n3. Complete the limited warranty registration card that is included in the carton and mail it to the Bose Customer Service address provided on the warranty card or the product packaging.\n4. If the product is covered by the warranty, Bose will repair or replace the product at no additional charge, except for shipping charges. Bose may require you to return the product to an authorized Bose® dealer.\n5. Warranty coverage begins on the date of purchase and ends 1 year later (or a longer period if required by local law).\n6. This limited warranty does not cover damage caused by accidents, misuse, normal wear and tear, or service performed by anyone other than Bose or an authorized Bose® dealer.\n7. Any implied warranties, including those related to merchantability or fitness for a particular purpose, are limited in duration to 1 year from the date of purchase.\n\nFor more information about this limited warranty, contact Bose Customer Service. Refer to the address list included in the carton.\n\n ',
 " This owner's guide provides important safety information and operating instructions for your Bose® SoundLink™ headphones. Please read it carefully before using your product. If you have any questions or concerns, contact the Bose Customer Support team. ",
 ' 5\n\nysi6u3\n\nUsiNG Your HEADSET\n\nWearing the headset (cont.)\n\nTo put on the headset:\n\n1. Hold the headband with one hand and the earcup with the other.\n2. Pivot the earcup up and over the top of your head, until it is in front of you.\n3. Slide the cushion over your ear, using your fingers to gently press the cushion against your head.\n4. Repeat for the other earcup.\n5. Lower the headband onto your shoulders. The headset should be comfortably snug around your ears and neck.\n\nUsing the microphone\n\n1. Hold the bottom of the microphone near the mouth, about 1/2 inch (about 1 cm) away from your mouth when speaking.\n2. Speak with confidence.\n3. Adjustment: If you encounter problematically相关的问题，请参用教训。\n\nUsiNG Your HEADSET\n\n以下列的不会出现：\n\n1.打滚式（例如）。\n2.找到正确的适用。\n3.完成。',
 ' .Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused.\n\nNote: The serial number is located on the warranty card included in the carton.\n\n16\n',
 ' The 3D glasses can also be charged using the USB charging AC power adapter, which is not included with the glasses. To charge the battery using this method:\n\n1. Turn off the 3D glasses.\n2. Plug one end of the USB charging cable into the connector for charge on the 3D glasses.\n3. Plug the other end of the USB charging cable into a USB port on the computer or an AC outlet (if using a USB power adapter).\n4. The battery starts charging and the yellow LED indicator light on the 3D glasses turns on.\n5. The battery charge will stop if you turn on the 3D glasses while charging the battery.\n6. Do not use any infrared communication device near the LCD screen. Follow these instructions:\n1 Remove covers"[1]',
 ' 1. Contact the Bose organization in your country/region to obtain a Return Material Authorization (RMA) number. Refer to the address list included in the carton for contact information.\n2. Complete the RMA form provided with your product registration card.\n3. Follow the instructions provided on the RMA form to package and ship the product. Be sure to include all parts of the product, including the power cable and remote control, as well as any packaging materials.\n4. Ship the package to the address indicated by the Bose organization in your country/region.\n5. Allow 10-15 business days for processing and shipping the returned product.\n\nLimited warranty coverage is as follows:\n\n* The limited warranty covers defects in material or workmanship for one year from the date of purchase.\n* Repair or replacement of a defective product will be made at the Bose organization\'s discretion.\n* This limited warranty does not cover damage caused by normal wear and tear, misuse, abuse, accident, improper use, or failure to follow instructions provided with the product.\n* To obtain repair or replacement service during the one-year limited warranty period, contact the Bose organization in your country/region. Refer to the address list included in the carton for contact information.\n\nCustomer service\n\nTo contact Bose Customer Service for help, or to replace a lost remote control, see the phone\nnumbers on the inside the back cover of your Wave" music system guide.\n\n16\n\n ', " This is a user manual for the Bose SoundLink USB key and adapter. The document provides information about the product's features, setup, and safety precautions. It also includes warranty and disposal instructions. The manual is intended for users in North America and contains additional safety information on the Important Safety Instructions sheet enclosed in the shipping carton. ",
 ' Sound.” Please be aware that using these additional features can impact the performance and audio quality of your Bose mobile on-ear headset. If you are unsure about the compatibility of any audio device with the Bose mobile on-ear headset, or if you need further assistance, please contact our customer service department at 1 (800) 379-4650.\n\nThank you for choosing the Bose” mobile on-ear headset. We hope that it provides you with many hours of enjoyment. If you have any questions or concerns about your product, please do not hesitate to contact our customer service department at 1 (800) 379-4650.\n\nBose Corporation® ',
 ' .Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number will be refused.\n\nNote: The serial number is located on the warranty card included in the carton.\n\n16\n``` \n\nToxic or hazardous substances or elements and their respective parts:\n\nName | Part Number | Contents of Toxic or Hazard Substances or Elements:\n```\n\nFor additional help in solving problems,see notes about the product at Owners.Bose.com.\n\nTo contact Bose Customer service,see phone numbers on theback of your Wave music system guide.\n\n\n15 \n\n你的Wave音系需要的这些:\n\n*要在空处中"\n\n*要在悰的地中找到"\n\n*要在20个地中寻到"\n\n*要在300个的地中找到"\n\n*要在500个的地中找到"\n\n*要在300个的地中找到"\n\n*要要在杊中使用的1"\n\n*要要在200个的地处找到"\n\n*要要在1200个的地中找到"\n\n*要要在500个的地中找到*"\n\n*需要的300个的地和潤的地"\n\n*需要要在800个的地处找到*"\n\n*需要你的Wave音系需要的200个的地，这些都是你的Wave音系的主要部分"\n\n*需要你的Wave音系的有效的捕"\n\n*需要你的Wave音系需要的1和2"\n\n*需要你的Wave音系需要的1和2"\n\n*需要你的Wave音系需要的300个的地"\n\n*需要你的Wave音系需要的1和2"\n\n*需要你的Wave音系需要的4"\n\n*如果你的Wave音系中发现问题，那么应该在500个的地处找到"\n``` \n\n\n15 \n\n\n*需要您的Wave音系的潤地和200个的地。"\n\n*"需要你的Wave音系中发现问题，如果在1200个的地处没找到."\n\n*-需要你的Wave音系中发现的捕"\n\n*需要你的Wave音系的有效的1和2"\n\n-需要你的Wave音系需要的1和2"\n\n*-需要您的Wave音系需要的300个的地"\n\n*-需要你的Wave音系中发现的捕"\n\n*-需要你的Wave音系需要的1和2"\n\n*-需要你的Wave音系需要的1和2"\n\n*需要你的Wave音系需要的1和2"\n\n-需要你的Wave音系中发现的问题，如果在1200个的地处没找到."\n\n*-需要你的Wave音系需要的1和2"\n\n*需要您的Wave音系需要的1和2"\n\n*需要你的Wave音系需要的1和2"\n``` ',
 ' ',
 ' ',
 ' Sound.” It is important to note that any third-party audio enhancements may impact the sound quality and performance of your Bose headset. While these features can provide additional options for personalizing your listening experience, they may not be fully compatible with your Bose headset.\n\nIf you are experiencing issues with your Bose headset or if you have any questions about using it with your mobile device, please contact Bose Customer Service. The address list included in the carton provides information on how to reach us.\n\nThank you for choosing Bose for your audio needs. We hope that you enjoy your new Bose headset and that it meets your expectations for performance and comfort.\n\n©200000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000100000000000000000000000000000000000000000000000000,108,274,164,164),(39,142,165,477)};\n\n  \\put(175,155){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(694,160){0.348,0.141,0.434,0.344};\n  \\put(175,235){0.348,0.141,0.434,0.344};\n  \\put(368,235){0.348,0.141,0.434,0.344};\n  \\put(537,234){0.348,0.141,0.434,0.344};\n  \\put(664666266266626662662662662662662662662662666266266266266266266266266266466626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626626619000000000000000000001150100230140140110101',
 ' How do I clean my headset?\n\nYou can wipe your headset with a soft dry cloth. Avoid using liquids or soaps on the earpieces or cushions, as they can damage the materials. If you need to remove any dirt or debris from the hinges or other parts of the headset, use a small brush or compressed air to gently clean the affected areas.\n\nCan other Bluetooth phone users hear my conversation?\n\nWhen you pair your headset to your Bluetooth phone, you are creating a private link between only these two devices. The wireless Bluetooth technology used in your headset is not easily monitored by third parties because Bluetooth wireless signals areep" ',
 ' Biphenyl] Dioxin | Flame retardants | Volatile Organic\nComposites (VOC) | Antimony trichloride | Nickel | Chlorofluorocarbons\n(CFCs) | Arsenic | Lead | ',
 " The image you've provided appears to be a screenshot of a user manual or guide for a pair of Bose mobile on-ear headset. The manual includes instructions on how to connect the headset to various devices, such as cell phones and audio devices, and provides information on battery life, charging time, and standby time. It also includes safety warnings and guidelines for cleaning and maintaining the headset. Additionally, it lists the specifications of the headset, including Bluetooth version, operating range, and talk time. ",
 ' Sound” or “Virtual Surround.” Be mindful of these various audio features, as they can significantly affect the audio performance and quality when used with your Bose” mobile on-ear headset. When in doubt, consult the owner’s guide for your particular audio device to ensure proper use of these settings.\n\nCaution: Overuse or reliance on these audio enhancement features may cause hearing damage or reduce sound quality over time. ', ' Bisphenol A) Flame Retardants | Dioxins & PCBs\nElectrical Components | Formaldehyde\nPlastic Parts | Lead\nHeadband | Mercury | Cadmium Hexavalent\nCushion Foam | Polybrominated Biphenyl\nEar Cup | Cadmium Hexavalent | Flame Retardants\nSpeaker | Lead | Polybrominated Biphenyl | Dioxins & PCBs\nAuxiliary Input Jack | Mercury | Cadmium Hexavalent | Flame Retardants\nCharging Connector | Formaldehyde | Flame Retardants | Dioxins & PCBs\n \nNames and Contents of Toxic or Hazardous Substances or Elements\nPart | Lead] Mercury | Cadmium Hexavalent | Polybrominated Biphenol| Flame Retardants| Dioxins & PCBs| Lead|Cadmium Hexavalent|Polybrominated Biphenol|Flame retardants|Dioxins & PCBs\nNames | _Polybrominated Biphenol| _Formalehyde| _Polybrominated Biphenol| Dioxins & PCBs| _Lead_| _Mercury_| _Cadmium Hexavalent_| _Polybrominated Biphenol_| _Flame retardants_ | _Dioxins & PCBs_| _Lead_| _Mercury_| _Mercury_| _Cadmium Hexavalent_| _Polybrominated Biphenol_| _Flame retardants_ | _Polybrominated Bip�子的名和内容_| _Lead_| _Mercury_| _Mercury_| _Mercury_| _Cadmium Hexaval�_| _Polybrominated Bip�子的名和内容_| _Formalehyде_ | _Polybrominated Bip�子的名和内容_ | _Lead_| _Mercury_| _Mercury_| _Cadm� ',
 ' 3D]” or “Stereo Wide” settings. Please note that using any of these features with your Bose” mobile on-ear headset may alter the audio performance and sound quality in a way not intended by the manufacturer, and may affect the overall sound experience. It is recommended to use these features at your own discretion. ',
 ' Biphenyl A (BPA) | Chlorobenzene | Chlorophenol | Naphthalene | Naphthalene | Perfluoroalkyls (PFAs) | PFOS | PFOA |\n\n ',
 ' .Bose.com for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton.\nCartons not bearing a return authorization number, where required, will be refused.\n\nCustomer service\n\nFor additional help in solving problems, see notes about the product at: Owners.Bose.com.\nTo contact Bose Customer Service for help, or to replace a lost remote control, see the phone numbers on the inside the back cover of your Wave" music system guide.\n\n16\ndern\n\n ',
 ' Services page on Bose.com for contact information) for assistance in obtaining a Return Merchandise Authorization (RMA). Please note that you may be responsible for return shipping charges.\n\n2. Pack the product securely and ship it to the address provided by the Bose Customer Service representative.\n\n3. Include your proof of purchase and, if applicable, the original packaging and accessories, including any missing parts.\n\nLimited Warranty coverage ends 1 year after the date of the original purchase or when you first own the product, whichever is later.\n\n ',
 ' ',
 ' What you must do to obtain Limited Warranty Service:\n\nReturn product, with proof of purchase from an authorized Bose dealer, using the following procedures:\n\n1. Contact the Bose organization in your country/region (visit Global.Bose.com for Bose contact information in your country/region) for specific return and shipping instructions; and\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused. ',
 ' ',
 ' Brominated | Diphenyl)\nFlame retardants | Phthalates\nElectrical components | Nickel | Chromium | Copper\n\n ',
 " The text is a bit small to read clearly, but I'll provide you with an overview of what it says:\n\n1. Introduction to Bluetooth technology and its range limits.\n2. Instructions for cleaning the headset using a soft, dry cloth.\n3. A note about the private link created between the headset and phone during pairing.\n4. Suggestions for adjusting the volume or moving to another location if there's an echo on a call.\n5. Information on pairing with other Bluetooth devices, emphasizing that it is difficult for third parties to monitor Bluetooth signals.\n6. A statement about the headset and travel adapter not being connected properly if they are separated during charging.\n7. A mention of UL certification for the travel adapter, and a reminder to follow safety instructions as outlined by UL guidelines.\n8. Information on Industry Canada certification and safety requirements.\n9. Safety warnings about proper connection to a power supply, orientation of the power unit, and avoiding physical abuse, modification, or repair by unauthorized third parties.\n10. The warranty information for the headset.\n\nPlease note that some specific details in the text might be difficult to read due to the image's resolution. ", ' ',
 ' The text in the image is too small to read clearly. ',
 ' Support on Bose.com for a list of Bose Customer Service contacts around the world) to obtain a Return Merchandise Authorization (RMA) number and instructions for returning your product. If you need assistance, please contact the Bose Customer Service organization in your country/region.\n\n1. Send the product, along with the RMA number and any other required documentation, to the address specified by Bose Customer Service. Shipping is at your expense. Please do not send the product to any other location without authorization.\n\n1. Keep your proof of purchase handy in case it is needed for warranty service.\n\nThis limited warranty gives you specific legal rights, and you may also have other rights which vary from country. ',
 ' Brominated Flame Retardants)\nBrominated Fluorocarbons\n\nNotes:\n\n1. This product contains one or more of the chemicals listed above in a concentration that is within the allowable limits under the regulations and guidelines set by the respective government agencies. For example, in Europe, this product is compliant with the RoHS Directive 2011/65/EU.\n2. None of the substances listed above are present in concentrations above the regulatory limits.\n3. This product is designed and manufactured to meet or exceed all applicable regulations and guidelines regarding toxic or hazardous materials, including but not limited to, the Restriction of Hazardous Substances (RoHS) Directive 2011/65/EU, the European Union Directive on Electrical Equipment 2014/35/EU, and the Waste Electrical and Electronic Equipment Regulations in Europe.\n4. This product is recyclable. Please check local regulations for recycling requirements and facilities. ',
 ' ',
 ' This document contains product specifications and safety instructions for the Samsung Bluetooth Headset. Please refer to the relevant sections for further details:\n\n1. Product Specifications\n* Headset: A Samsung Bluetooth headset.\n*Profiles: The Samsung Bluetooth headset supports the Simple Pang" ',
 ' The image shows a Bluetooth headset and its packaging with some specifications printed on it. The headset is designed to work with Bluetooth-enabled devices and supports the Headset Profile (HSP) for hands-free use. It has a range of features that they find so." ',
 ' 3D] sound.” Your Bose® mobile on-ear headset is designed to work well with the audio output from any device. However, using some of these enhanced features may require adjustments in the way you listen to music. These adjustments can help improve your listening experience with your Bose®\n\nUsingWithout',
 ' ',
 ' The image you provided shows an electronic device with various buttons and ports, including a screen displaying the home menu, a USB port, and a camera lens. There is also text that says "Bose," "Wave," and "Customer service." Additionally, there are instructions for using the device, such as connecting it to a computer or pairing it with other devices via Bluetooth. ', ' Using Your Mini 3i Smartphone\n\nConnecting a Bluetooth Headset\n\n1. Turn on your Bluetooth headset.\n\n2. On your smartphone, go to Settings > Bluetooth.\n\n \n\n3. If Bluetooth is not already enabled, tap the switch next to "Bluetooth" to turn it on.\n\n4. Once your headset appears in the list of available devices on the screen, tap the name of your Bluetooth headset to connect it to your smartphone.\n\n \n\n5. If prompted, enter the passkey displayed on your smartphone\'s screen on your headset. Thepasskey is a',
 ' Sound” or “virtual surround sound.” When using your Bose® mobile on-ear headset with these devices, you can choose to activate these features to enhance your listening experience. However, keep in mind that some of these features may alter the natural audio balance and tonal quality provided by the Bose® mobile on-ear headset.\n\nUsiNG Your HEADSET\n\nUsing the headphone cord with inline microphone\nTo answer a call while using your Bose® mobile on-ear headset, simply push the button on the in-line microphone during the call. To hang up, either push the button again or pull down on the microphone to end the call and return to music playback.\n\nUsiNG Your HEADSET\n\nCare & Maintenance\n\nClean the earpieces using a soft cloth dampened with a gentle cleaning solution or mild soap and water. Avoid using harsh chemicals or abrasive materials that may damage the earpiece cushions or the headband. To clean the microphone, use a soft brush or compressed air to remove dust and debris.\n\nNote: If you find it difficult to keep your Bose® mobile on-ear headset from tangling, consider using a twist tie or cable wrap solution.\n\nUsiNG Your HEADSET\n\nTroubleshooting\n\nIf you experience any issues with your Bose® mobile on-ear headset, try the following troubleshooting steps:\n\n• Ensure that the headphone cord is connected properly to both the headset and your mobile phone or audio device.\n\n• Check for any obstructions in the headphone cord.\n\n• If using an adapter with your mobile phone, make sure it’s the correct one for your particular phone model.\n\n• Verify that the volume on your mobile phone or audio device is turned up and is not muted.\n\n• Inspect the earpiece cushions for any damage or debris that may be affecting sound quality.\n\n• If you continue to experience issues, contact our Customer Service Team for assistance at 1-800-444-2673.\n\nUsiNG Your HEADSET\n\nLimited Warranty\n\nBose® mobile on-ear headset is backed by a limited warranty against defects in materials and workmanship under normal consumer use for one year from the date of purchase. For more information, visit our Customer Service page at www.bose.com/support or call 1-800-444-2673.\n\nThank you for choosing Bose® mobile on-ear headset. Enjoy your music and calls! ',
 ' Biphenyl A Brominated Flame Retardant)\n\n1. Screen (LED, LCD, etc.) ',
 ' This appears to be a product manual for a Bose mobile on-ear headset. The document provides instructions on how to use the headset with various devices, such as mobile phones or audio players. It also includes information about the warranty and safety guidelines for the product. ',
 ' Sound” or “3D Audio”. If you are experiencing difficulties with the audio quality while using the headset, it is possible that the “enhancement” feature settings of your device may be affecting the performance of the headset. It is recommended that you check and adjust these settings to see if they can improve the sound experience.\n\nIf you continue to have issues, contact Bose Customer Service for help.\n\nCustomer service phone numbers are listed on the inside back cover of your Wave" music system guide. ',
 ' 14\nysi6u3\n',
 ' .Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number will be refused.\nNote: The serial number is located on the warranty card included in the carton.\n\n ',
 ' .Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused.\nNote: The serial number is located on the warranty card included in the carton.\n\n ',
 ' 11\n\nTroubleshooting (continued)\n\nProblem What to do\nConnecting remains "NO SIGNAL" on the Wave® system display and the adapter light blinks continuously.\n\n* Check your computer for any software updates or new drivers that may be required.\n\n* Restart your computer and try connecting again.\n\n* If you\'re using a USB hub, make sure it is powered and the SoundLink™ USB key is connected to it.\n\n* Try connecting the Wave® music system directly to a free USB port on your computer.\n\n* Verify that the SoundLink™ USB adapter is properly seated in the Wave® music system.\n\n* Check if other devices are connected to your computer, this could be interfering with the SoundLink™ USB key.\n\nIf none of the above troubleshoots resolve the issue, please contact Bose Customer Service for further assistance. Contact list included in the carton. ',
 ' .Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused.\n\nNote: The serial number is located on the warranty card included in the carton.\n\n16\n\nBose SoundLink™ Adapter and USB Key Specifications\n\nModel number: 720595 (for Mac® computers) or 738595 (for Windows® computers)\n\nPower Supply\n\nVoltage: 100-240 V~, 50/60 Hz\nCurrent: 0.3 A\n\nFrequency response: 100 kHz\n\nType: Internal combustion engine or battery\n\nOperating temperature range: 0° to 70° C (32° to 158° F)\n\nStorage temperature range: -20° to 65° C (-4° to 149° F)\n\nPower consumption: Less than 2.5 W\n\nBattery life\n\nCharge time: Approximately 3 hours\n\nDimensions (adapter):\n\nHeight: 78 mm (3 1/8")\nWidth: 60 mm (2 3/8")\nDepth: 29 mm (1 1/8")\n\nWeight (adapter): 45 g (1.6 oz)\n\nDimensions (USB key):\n\nHeight: 55 mm (2 1/4")\nWidth: 29 mm (1 1/8")\nDepth: 10 mm (3/8")\n\nWeight (USB key): 27 g (1 oz)\n\nSystem Compatibility\n\nCompatible with Windows® 7, Vista® and XP® operating systems\n\nCompatible with Mac OS® X v.10.5 or later\n\nCompatible with Internet radio services available on the Internet\n\nNote: Software requirements may vary by music service provider.\n\nBose® is a registered trademark of Bose Corporation.\n\nApple®, Macintosh® and macOS® are trademarks of Apple Inc., registered in the U.S. and other countries.\n\nMicrosoft®, Windows® and the Windows logo are trademarks, or registered trademarks of Microsoft Corporation in the United States and/or other countries.\n\n17\n\nConnection range for SoundLink™ Adapter and USB Key\n\nThe SoundLink™ Adapter and USB Key can operate with the Wave® music system up to a line-of-sight distance, which is typically around 30 feet (9 meters) in an unobstructed indoor environment. If you are experiencing reception problems or the connection range appears to be limited, try moving the SoundLink™ adapter farther from the Wave® music system or changing its orientation (for example, flipping it horizontally).\n\nFor further assistance, contact Bose Customer Service. Refer to the phone numbers on the inside back cover of your Wave® music system guide.\n\n18\n\nBose SoundLink™ Kit User Guide\n\nThis guide contains information about the features and functions of your Bose SoundLink™ kit. You can find answers to frequently asked questions, tips for getting the most out of your kit, and troubleshooting assistance in case you encounter problems.\n\n1. Connecting the SoundLink™ Adapter and USB Key:\nTo connect the SoundLink™ adapter and USB key to your computer, open the protective cover on the USB key, press the button on the bottom of the key for more than 3 seconds until the power light begins to flash. When the light changes from flashing to solid and the Wave® music system emits a beep, linking is complete.\n2. Selecting a Source:\nYou can select the computer as the source by pressing the SoundLink/AUX button on the remote control until COMPUTER is displayed. You can also use your computer\'s media player or Internet radio application to play music and control volume. Note: Not all players and services support this form of playback control.\n3. Adjusting Volume:\nTo adjust volume using the remote control, press the + and - buttons until the desired level is reached. You can also use your computer\'s volume controls to adjust the volume.\n4. Controlling Playback:\nUse the remote control\'s play/pause and seek buttons to control playback on your computer. Note: Not all players and services support this form of control.\n5. Battery Life:\nThe SoundLink™ adapter has a battery life of approximately 3 hours, which should be sufficient for most listening sessions. If you find that the battery life is inadequate, consider using an additional rechargeable battery pack or powering the adapter with an external power source (not included).\n6. Troubleshooting:\nIf you are experiencing reception problems or other difficulties with your SoundLink™ kit, contact Bose Customer Service for assistance. Refer to the phone numbers on the inside back cover of your Wave® music system guide.\n7. Limited Warranty:\nYour Bose SoundLink™ Kit is covered by a limited warranty against defects in material and workmanship for one year from the date of purchase. This warranty does not cover damage caused by normal wear and tear, or damage resulting from abuse, misuse, accident, or failure to follow instructions for use and care. For warranty service, contact Bose Customer Service. Refer to the phone numbers on the inside back cover of your Wave® music system guide.\n8. Disposal:\nBose is committed to protecting the environment. If you need to dispose of your SoundLink™ kit, follow local recycling guidelines for electronic waste. Some municipalities offer special programs for collecting and recycling used electronics. Check with your local government or waste management facility for more information.\n\n19\n\nBose® SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose Corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n20\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose Corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n21\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose Corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n22\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose Corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n23\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose Corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n24\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose Corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n25\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose Corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n26\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n27\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n28\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n29\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n30\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n31\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n32\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n33\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n34\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n35\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n36\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n37\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n38\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n39\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n40\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n41\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n42\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n43\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n44\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n45\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n46\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n47\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n48\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n49\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n50\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n51\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n52\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n53\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n54\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n55\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n56\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n57\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n58\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n59\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n60\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n61\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n62\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n63\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n64\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n65\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n66\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n67\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n68\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n69\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n70\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n71\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n72\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n73\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n74\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n75\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n76\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n77\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n78\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n79\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n80\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n81\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n82\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n83\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n84\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n85\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n86\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n87\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n88\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n89\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n90\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n91\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n92\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n93\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n94\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n95\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n96\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n97\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n98\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n99\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.\n\n100\n\nFor customer service inquiries:\nIn the U.S.: 1-800-659-7535\nOutside the U.S.: +1-617-782-4425\nwww.bose.com/support\n\nBose SoundLink™ Kit User Guide\n\nThank you for choosing Bose to enhance your computer audio experience. If you have any questions or concerns about your kit, please feel free to contact Bose Customer Service. Our representatives are available to help you get the most out of your new SoundLink™ kit.\n\nBose corporation\nOne Bose Way\nEden Prairie, MN 55344\nU.S.A.\nwww.bose.com\n\nPlease recycle your packaging and other waste materials to help preserve our environment.', ' The text provides information about the product and its usage. It includes details such as the warranty, specifications, and instructions for pairing and connecting the headset to a Bluetooth device. There are also answers to some questions that may arise during the use of this product. ',
 ' .Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused.\n\nNote: The serial number is located on the warranty card included in the carton.\n\n16\n',
 ' Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number will be refused.\n\nNote: The serial number is located on the warranty card included in the carton.\n\n16\n` ',
 ' 11. Troubleshooting (cont.)\n\nWhat to do\n\nIf you experience any trouble using your headphones, try the following troubleshooting instructions:\n\nSymptom A: SOUNDLINK or COMPUTER never appear on the Wave® system display and no lights blink.\n\n* Ensure that both ends of the audio cable are securely connected.\n* Make sure the computer is turned on and that its volume control is set to maximum.\n* Make sure you have downloaded and installed the SoundLink™ software for your operating system.\n\nIf this does not resolve the problem, contact Bose Customer Service. Refer to the address list included in the carton.\n\n12. Auxiliary Ports (cont.)\n\nWhat to do:\n\nSymptom A:\nSOUNDLINK or COMPUTER never appear on the Wave® system display and no lights blink continuously.\n= NO SIGNAL - appears eventually on the system. display.\n\nOr, see Symptom B on the next page.\n\n* Press and hold the VOLUTE button until the volume level becomes 2 Pp.\n* Ensure that you are using a Wave® USB adapter (included with your kit) to connect your SoundLink™ to the system.\n* Make sure that your computer is turned on and its audio output port is connected to the Wave® music system.\n\nIf this does not resolve the problem, contact Bose Customer Service. Refer to the address list included in the carton.\n\n13. Auxiliary Ports (cont.)\n\nWhat to do:\n\nSymptom B:\nCONNECTING remains on the Wave® system display and the adapter light blink continuously.\n= NO SIGNAL - appears eventually on the system. display.\n\nOr, see Symptom A on the previous page.\n\n* Ensure that you are using a Wave® USB adapter (included with your kit) to connect your SoundLink™ to the system.\n* Make sure that your computer is turned on and its audio output port is connected to the Wave® music system.\n* If you received an upgrade CD with your kit and have not used it, be sure to run it now.\n\nIf this does not resolve the problem, contact Bose Customer Service. Refer to the address list included in the carton.\n\n14. Auxiliary Ports (cont.)\n\nWhat to do:\n\nSymptom B:\nCONNECTING remains on the Wave® system display and the adapter light blink continuously.\n= NO SIGNAL - appears eventually on the system. display.\n\nOr, see Symptom A on the previous page.\n\n* Ensure that you are using a Wave® USB adapter (included with your kit) to connect your SoundLink™ to the system.\n* Make sure that your computer is turned on and its audio output port is connected to the Wave® music system.\n* If you received an upgrade CD with your kit and have not used it, be sure to run it now.\n\nIf this does not resolve the problem, contact Bose Customer Service. Refer to the address list included in the carton.\n\n15. Auxiliary Ports (cont.)\n\nWhat to do:\n\nSymptom B:\nCONNECTING remains on the Wave® system display and the adapter light blink continuously.\n= NO SIGNAL - appears eventually on the system. display.\n\nOr, see Symptom A on the previous page.\n\n* Ensure that you are using a Wave® USB adapter (included with your kit) to connect your SoundLink™ to the system.\n* Make sure thatyour computer is turned on and its audio输出口是连接到theWave®音乐系统。\n* If您收到一个升级的CD，并未使用它，请确保运行它。\n\n如果这样做不能解决问题，请联系Bose客户服务。参考地址表中包含的联系地址列表。 ',
 ' Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently ‘on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused.\nNote: The serial number is located on the warranty card included in the carton.\n\nThe warranty for this product is valid in countries where Bose products are sold. Any repairs or replacements provided under the limited warranty will be made with new or reconditioned parts as determined by Bose, and any service performed will be done using Bose authorized service personnel or facilities.\n\nThis limited warranty does not cover damage caused by normal wear and tear, misuse, abuse, neglect, modification, accident, improper handling, installation in an unsuitable location, exposure to extreme temperatures, or other acts of God or the public enemy.\n\nBose reserves the right to repair, replace or dispose of any defective product, at its sole discretion, if the above warranty conditions are not met. Any implied warranties are limited in duration to the duration of this limited warranty. Some jurisdictions do not allow limitations on duration of implied warranties so the duration of such warranties may be longer.\n\nThis limited warranty gives you specific legal rights. You may have other rights under the laws of your country or region.\n\n16\n',
 " the computer audio output to be directed to the Wave® system. Check the owner's guide for instructions on how to do this with your particular computer.\n\nIf you are using a Windows computer and need additional assistance, please refer to the Bose® SoundLink™ software download at Global.Bose.com/SoundLink. ",
 " Please note that the above instructions are for a specific product and may not apply to all Bose products. Always follow the manufacturer's instructions for your specific product. ",
 ' The Bose Powerfoot Bagless Vacuum Cleaner is designed to provide you with the best possible cleaning experience. However, as with any vacuum cleaner, normal wear and tear can occur over time. Here are some common issues that may arise and what you can do to resolve them:\n\n1. No suction or audio in one earcup only: Ensure that both ends of the audio cable are securely connected. If this persists, try a different audio device.\n2. Distorted bass: Make sure both earcup cushions are firmly seated. The high-density foam will regain its shape after a few minutes in a neutral position.\n3. Crack or hole in the Hose: Broken or worn Drive Belt - Replace Drive Belt.\n4. Rotating Floor Brush bristles worn: Replace Brush.\n5. Rotating Floor Brush jammed: Clear clog, see page 8-9.\n6. Clog in vacuum: Empty Dust Bin.\n7. Dust Bin full: Empty Dust Bin.\n8. Dust Bin not installed properly: Position correctly and lock in place. Check Pre-motor and Post-motor filters.\n9. Visible dirt escaping from cleaner: Position correctly and lock in place. Check Pre-motor and Post-motor filters. Maintaining Mini Turbo Brush, see page 11.\n\nIf any problem persists, contact Bose Customer Service for return and shipping instructions. Please refer to the warranty card included in the carton for details on how to register your product. Failure to register will not affect your limited warranty rights. The serial number is located on the warranty card. ',
 ' ',
 " Sonvnanal Nivin £0.25\n\nmaintenance\\_12\n\nTroubleshooting\n\nCheck this list of possible problems and solutions before bringing your vacuum to a service center.\n\n13,\n\nPROBLEM\nVacuum cleaner won't run\n\n* Power cord not plugged in\n* Blown fuse/tripped breaker in home electrical system\n\npossible solution\n\n* Check electrical plug\n* Replace fuse or reset breaker\n\n‘Vacuum cleaner and tools won't pick up dirt\n\n* Incorrect Height Adjustment\n\n* Hose not attached to suction opening\n\n* Twist Snap Hose not securely attached\n\n* Crack or hole in Hose\n\n* Broken or worn Drive Belt\n\n* Rotating Floor Brush bristles worn\n\n* Rotating Floor Brush jammed\n\n* Clog in vacuum\n\n* Dust Bin full\n\n* Dust Bin not installed properly\n\n* Filters are dirty\n\n14,\nMini Turbo Brush is clogged or debris is wrapped around the brush roll\n\npossible solution\n\n* Cut off pet hairs or threads surrounding brush with scissors\n\n15,\nVisible dirt escaping from cleaner\n\npossible solution\n\n* Dust Bin full\n\n+ Filters missing or installed incorrectly\n\n "]

In [ ]:
print(len(saved))

85


In [ ]:
answers9 = []

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[14:], start=15):
    if sublist_index > 15:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers9.append(answer)

Index: 0
How to receive more complete service about this product?
 To reset the SoundLink™ USB key to its factory settings, follow these steps:

1. Ensure that all power is disconnected from the adapter and the remote control, if connected.
2. Press and hold the Power button for 10 seconds or until the green light flashes rapidly. This will cause the adapter to restart in the default factory mode.
3. Connect the USB key to your computer using the SoundLink™ USB key cable. The connection process begins automatically.
4. When prompted, follow the on-screen instructions to authorize and pair the SoundLink™ USB key with your computer. This will restore the original factory settings of the USB key.

TROUBLESHIPM

KANNICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH Rürt,

KANN NICH RÜRT NICH RÜRT NICH RÜрт NICH RÜrt NICH RÜrt NICH Rü rt,

KANN NICH RÜRT NICH RÜRT NICH Rü rt,

KANN NICH R

In [ ]:
answers9

[' To reset the SoundLink™ USB key to its factory settings, follow these steps:\n\n1. Ensure that all power is disconnected from the adapter and the remote control, if connected.\n2. Press and hold the Power button for 10 seconds or until the green light flashes rapidly. This will cause the adapter to restart in the default factory mode.\n3. Connect the USB key to your computer using the SoundLink™ USB key cable. The connection process begins automatically.\n4. When prompted, follow the on-screen instructions to authorize and pair the SoundLink™ USB key with your computer. This will restore the original factory settings of the USB key.\n\nTROUBLESHIPM\n\nKANNICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH RÜRT NICH Rürt,\n\nKANN NICH RÜRT NICH RÜRT NICH RÜрт NICH RÜrt NICH RÜrt NICH Rü rt,\n\nKANN NICH RÜRT NICH RÜRT NICH Rü rt,\n\nKANN NICH RÜRT NICH Rürt NICH RÜRT NICH Rü rt,\n\nKANN NICH 

In [ ]:
answers10 = []

In [ ]:
splitted_questions[15:16]

[['How to prevent the brush stops rotating?',
  'What if the suction is weak although the dust bin is empty?',
  'How to replace the post-motor filter?',
  'What if I properly install the Rotating Floor Brush?',
  'What should I do after removing a screw?']]

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[15:], start=16):
    if sublist_index > 16:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers9.append(answer)

Index: 0
How to prevent the brush stops rotating?


OSError: [Errno 36] File name too long: '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAD1ASsDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3+iiigAooooAKKKKACkPSlooAbwByM03IPqKeBijHPWgChcabZ3xD3VrHLj7pYZNWY444QscabV7ADgVNjFGPegBaKKKAI8EEgAAGjGBjP50/bSbR35oAz7jQ9OuWZpb

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[16:], start=17):
    if sublist_index > 18:
        break
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers10.append(answer)

Index: 0
What if I place conductive objects on the top of the controller?


 simulated 3D while using the 3D function, check that the following items are correctly set up:

1. Ensure that your computer is connected to the 3D-capable display with an HDMI cable.
2. Ensure that you are wearing the correct 3D glasses specified by the TV manufacturer.
3. Make sure that you have not turned off the simulated 3D function on the computer.
4. Check if there are any interruptions in the power supply to the connected display or your computer.
5. If necessary, try restarting both the computer and the display.
6. If the issue persists, check that the 3D-capable display is functioning correctly according to the manufacturer's instructions.

If images are displayed in a distorted manner while using the simulated 3D function, try adjusting the 3D effect settings using the VAIO Control Center. 
Index: 1
How to avoid injury from tip-over?
 1. Unplug the Wave® system from the power supply before connecting it to the SoundLink™ kit.
2. Connect the SoundLink™ kit to your Wave® syst

KeyboardInterrupt: 

In [ ]:
print(len(answers10))

2


In [ ]:
answers11 = []

In [ ]:
for sublist_index, sublist in enumerate(splitted_questions[18:], start=19):
    for index, question in enumerate(sublist):
        print(f"Index: {index}")
        print(question)
        answer = chain.invoke(question)
        print(answer)
        answers11.append(answer)

Index: 0
What is the 3D function?


KeyboardInterrupt: 

In [ ]:
final = answers + answers2 + answers3 + answers4 + answers5 + answers6 + answers7 + answers8 + answers9 + answers10 + answers11
print(len(final))

115


In [ ]:
with open("answers.txt", "w") as file:
    # Iterate through each answer in the 'answers' list
    for answer in final:
        # Write each answer to the file
        file.write(f"'{answer}\n',")

In [ ]:
answersold = [' ', ' The image shows the instructions for a Bose SoundLink headset, which are primarily related to using the headset with a computer and connecting it via Bluetooth. The instructions also cover how to troubleshoot issues such as no signal or poor sound quality while on a call, and how to clean and charge the headset. Additionally, there is information about the FCC and Industry Canada certification, and the UL certified travel adapter for the headset. ', ' connection between the headset and your phone. This ensures that only your phone can connect to the headset.\n\n1. Turn on your phone and open the Bluetooth settings menu.\n2. Activate the Bluetooth feature on your phone.\n3. Search for available devices. Your headset will appear as "Bose® Headset".\n4. Select "Bose® Headset" from the list of available devices.\n5. Follow the on-screen instructions to complete the pairing process.\n6. Once paired, your phone and headset are automatically connected whenever you turn on the headset.\n\ni ‘Using the headset\n\n1. To answer an incoming call, press the Talk button once.\n2. To end the call, press the Talk button again or press and hold it for a few seconds.\n3. While on a call, to switch between the phone speakerphone and the headset microphone, press the Talk button.\n4. To use your headset to play music, navigate to the music player on your phone and press the Play/Pause button or Volume buttons on the headset.\n5. Press the Power button to turn the headset off.\n\ni ‘Disconnecting the headset\n\nTo disconnect the headset from your phone, simply turn it off. The connection will be automatically severed.\n\n ', " Your Bose” mobile on-ear headset has two 3.5mm jacks: one for your phone and another for an audio device. When using a 2.5mm adapter with your phone, the 3.5mm jack is used as the connection to the headset's cord. When using an audio device or if your phone does not have a 3.5mm jack, you can use one of the four included adapters.\n\nConnecting your phone:\n1. If your mobile phone has a 3.5mm jack, simply plug in one end of the headset cord to your phone and the other end to the headphone's cord.\n2. If your mobile phone does not have a 3.5mm jack, choose an adapter that is compatible with your phone model.\n3. Insert the adapter into your phone's charging port or audio jack and plug in one end of the headset cord to the adapter.\n4. The indicator light on your headset will flash four times to confirm a successful connection.\n\nConnecting your audio device:\n1. Plug one end of the headset cord into the 3.5mm jack of your audio device.\n2. The indicator light on your headset will flash four times to confirm a successful connection.\n3. To use the inline microphone, hold it near your mouth during phone calls or while using voice commands.\n4. To activate and deactivate your headset, slide the power switch {B} located on the side of the headband. The indicator light will change colors to indicate the battery level and charging status.\n\nWhen not in use, fold the earcups and wrap the headband around the headset for easy storage in the included carrying case. ", ' risk of electric shock, do not immerse the headset or travel adapter in water or other liquids.\n\nDO NOT drop the headset into water or use it with water, such as showering or bathing.\n\nDO NOT attempt to repair the headset yourself. If it is damaged, consult a professional for assistance.\n\nDo not wrap the headset cord around objects or in cords to prevent tangling, as it can create a hazard when unwrapped.\n\nDo not leave the travel adapter plugged in and unattended while charging. Unplug it after use.\n\nKeep the travel adapter away from heat sources or direct sunlight, as it may overheat.\n\nKeep the travel adapter away from water sources such as sinks, bathtubs, or showers to prevent damage to the electrical components.\n\nKeep the travel adapter away from any flammable materials or substances.\n\nDo not use the headset or travel adapter if it has been damaged or is malfunctioning in any way. Consult a professional for assistance.\n\n16\n\nDispose of the headset and travel adapter according to local recycling regulations. Do not dispose of them in the trash.\n\nCAUTION:\n\n* The headset and travel adapter should be kept out of reach of children to prevent accidents or injuries.\n\n* Keep the headset and travel adapter away from pets who may chew on the cords.\n\n* Use caution when using the headset and travel adapter in areas with high humidity levels, as it can cause damage to the electrical components.\n\n* Do not use the headset or travel adapter if you experience any discomfort or pain while wearing them. Consult a professional for assistance.\n\n* Keep the headset and travel adapter away from sources of dust, debris, or other foreign materials that may cause damage or malfunction.\n\nThank you for taking the time to read and understand these important safety instructions. If you have any further questions or concerns, please do not hesitate to contact us. ', ' Electrical shock is a risk with any electrical appliance. Do not immerse the headset or its cord in water or other liquids. Keep the headset away from sinks, bathtubs, or any sources of water to avoid accidental immersion.\n\nDo not use your headset near a swimming pool, hot tub, or while taking a shower or bath. Do not expose your headset to rain, snow, or any other source of moisture.\n\nAvoid using the headset in areas where it may come into contact with water, steam, or excessive humidity. Do not use the headset near a waterfall, fountain, lake, ocean, or any other body of water.\n\nIf you accidentally get your headset wet, unplug the device immediately and allow it to dry completely before reconnecting it for use.\n\nDo not use your headset in areas where it may come into contact with snow, ice, or freezing temperatures. Avoid using the headset in extremely cold conditions or leaving it exposed to extreme cold for extended periods of time.\n\nIf you accidentally expose your headset to snow, ice, or freezing temperatures, unplug the device immediately and allow it to dry completely before reconnecting it for use.\n\nDo not use your headset near a fire or any open flame. Avoid using the headset in areas where it may come into contact with an open flame or any source of excessive heat.\n\nIf you accidentally expose your headset to a fire or an open flame, unplug the device immediately and allow it to cool completely before reconnecting it for use.\n\nDo not use your headset near a radiator or any other source of excessive heat. Avoid using the headset in areas where it may come into contact with a radiator or any source of excessive heat for extended periods of time.\n\nIf you accidentally expose your headset to a radiator or any other source of excessive heat for extended periods of time, unplug the device immediately and allow it to cool completely before reconnecting it for use.\n\nDo not use your headset near an open window or any other source of draft or strong winds. Avoid using the headset in areas where it may come into contact with an open window or any source of draft or strong winds for extended periods of time.\n\nIf you accidentally expose your headset to an open window or any other source of draft or strong winds for extended periods of time, unplug the device immediately and allow it to dry completely before reconnecting it for use.\n\nDo not use your headset near a power outlet or any other electrical appliance that may come into contact with water. Avoid using the headset in areas where it may come into contact with a power outlet or any other electrical appliance that may come into contact with water for extended periods of time.\n\nIf you accidentally expose your headset to a power outlet or any other electrical appliance that may come into contact with water for extended periods of time, unplug the device immediately and allow it to dry completely before reconnecting it for use.\n\nDo not use your headset near a construction site or any other area where it may come into contact with hazardous materials. Avoid using the headset in areas where it may come into contact with hazardous materials for extended periods of time.\n\nIf you accidentally expose your headset to a construction site or any other area where it may come into contact with hazardous materials for extended periods of time, unplug the device immediately and allow it to dry completely before reconnecting it for use. ', ' This is an image of a product manual for the Bose SoundLink Audio Headphones, which appears to be for a computer audio system that connects wirelessly to a computer via a USB adapter. The manual provides troubleshooting tips and instructions for using the product, as well as information on the limited warranty included with the headphones. ', ' ', " The Bose OE2 audio headset is designed for use with many music-enabled mobile phones, including the Mini 3i smartphone. To find out which adapter to use with your phone, please consult the owner's manual that came with your phone.\nSafety Information: Please read this owner's guide carefully before using your new headphones. The Bose OE2 audio headset is not compatible for use with airplane seat jacks as it may result in personal injury due to overheating. Always be aware of your surroundings and avoid using headphones when operating a motor vehicle or where the inability to hear outside sounds may present a danger to you or others.\nConnecting your headset to your phone: Please refer to the owner's manual for your mobile phone for instructions on connecting the Bose OE2 audio headset to your phone.\nHeadphone cord with inline microphone: The Bose OE2 audio headset includes an inline microphone that allows you to make and receive calls while listening to music or other audio content. To use the inline microphone, simply connect the headset to your phone using the 3.5mm to 2.5mm mobile phone adapters.\nStoring your headphones: When not in use, the Bose OE2 audio headset folds up for easy storage in the included carrying case. To fold up the headphones, simply pull the earcups out and away from each other until they snap into place. The headband can be further folded for even easier storage.\nCleaning your headphones: The Bose OE2 audio headset does not require scheduled cleaning, but if cleaning is necessary, simply wipe the outside surfaces with a soft cloth. Be sure the earcup ports are kept clear, and that no moisture is allowed to get inside the earcups.\nReplacement parts and accessories: If you need replacement parts or accessories for your Bose OE2 audio headset, such as the carry case, earcup cushions, or battery, please contact Bose Customer Service for assistance. ", ' 1 Headphone cord\n2 Audio cable\n3 Ear cup cushion\n4 Carry case '," The Bose® OE2 audio headphones are covered by a two-year warranty from the date of purchase against defects in materials and workmanship. Please see the warranty card that came with your product for complete details.\n\nWarranty Repair: In the event that you experience any issues with the Bose® OE2 audio headphones, please contact the place of purchase for instructions on how to obtain a repair or replacement.\n\nDisposal: Do not dispose of electronic products in household trash. Recycle your electronic products at a local recycling center or through your city's recycling program. For more information about electronic product recycling and to find a recycling location near you, visit www.electronicstakeback.com or call 1-800-323-6757.\n\nBose is committed to minimizing its impact on the environment. It recycles over 94% of its waste materials and works hard to reduce the environmental impact of its products throughout their entire life cycle. For more information, please visit www.bose.com. Thank you for choosing Bose® audio headphones.\n\nYour satisfaction is important to us, so if you have any questions or concerns, please do not hesitate to contact Bose Corporation. Please save this owner's guide for future reference. ", ' This is a safety manual for the Bose CORPORATION SoundLink™ Bluetooth® Headset. Please follow the instructions below to ensure safe use of this product.\n\n1. Read these instructions before using the headset.\n2. Keep the headset and its accessories out of reach of children.\n3. Do not immerse the headset or its accessories in water or other liquids.\n4. Avoid contact with chemicals that can damage the material or plastic parts of the headset.\n5. Do not use the headset while swimming or showering.\n6. Keep the headset away from heat sources, such as direct sunlight or a stove.\n7. Do not drop the headset on a hard surface, as it may break.\n8. Avoid touching the metal parts of the headset with your hands when adjusting the headset.\n9. Do not use the headset for any other purpose than its intended function as a Bluetooth® headset.\n10. Keep the headset and its accessories away from pets, such as cats or dogs, as they may chew on them.\n11. Do not attempt to repair the headset yourself if it becomes damaged. Contact Bose CORPORATION for assistance.\n12. Dispose of the headset and its packaging according to your local regulations for electronic waste.\n\nImportant note: This equipment has been tested and found to comply with the limits for a Class B digital device pursuant to Part 15 of the FCC Rules. These limits are designed to provide reasonable protection against harmful interference in a residential installation. However, there is no guarantee that interference will not occur in a particular installation. If this equipment does cause interference to radio or television reception, which can be determined by turning the equipment off and on, the user is encouraged to try one or more of the following measures:\n\n* Reorient or relocate the receiving antenna or the headset.\n* Increase the separation between the equipment and receiver.\n* Connect the equipment into an outlet that is on a circuit separator from that to which the receiver is connected.\n* Contact Bose CORPORATION for further assistance. ', ' ', ' Sure, I can help you with that. Would you like to know more about a specific aspect of the Mini 3i smartphone or its features? ', ' , please refer to the compatibility chart located on our website at www.bose.com.au/compatibility.\n\n1. Turn off your mobile phone.\n2. Attach the 3.5mm end of the headset cord to the audio jack on your mobile phone or\naudio device.\n3. If you are using an adapter, attach the appropriate adapter to the headset cord before\nattaching it to your mobile phone or audio device.\n4. Turn on your mobile phone.\n5. Adjust the volume and playback controls as desired.\n6. Disconnect the 3.5mm end of the headset cord from your mobile phone or audio device.\n7. Fold the headphones back into their original position, then attach the carrying case to the headband for easy storage.\n\nConnecting your Bose” mobile on-ear headset to a computer:\n\n1. Turn off your computer.\n2. Attach the 3.5mm end of the headset cord to the audio jack on your computer.\n3. Turn on your computer.\n4. Adjust the volume and playback controls as desired.\n5. Disconnect the 3.5mm end of the headset cord from your computer.\n6. Fold the headphones back into their original position, then attach the carrying case to the headband for easy storage.\n\nConnecting your Bose” mobile on-ear headset to a home stereo or other audio equipment:\n\n1. Turn off your home stereo or audio equipment.\n2. Attach the 3.5mm end of the headset cord to the auxiliary input jack on your stereo or audio equipment.\n3. Turn on your home stereo or audio equipment.\n4. Adjust the volume and playback controls as desired.\n5. Disconnect the 3.5mm end of the headset cord from your home stereo or audio equipment.\n6. Fold the headphones back into their original position, then attach the carrying case to the headband for easy storage.\n\n ', " RISK OF ELECTRIC SHOCK DO NOT OPEN OR SERVICE THIS APPLIANCE WHILE IT IS PLUGGED IN AND SWITCHED ON.\n\n* Ensure your hands are dry before handling this appliance.\n* Do not immerse the travel adapter in water or other liquids.\n* Keep this appliance at least 6 feet away from any water source.\n* Never touch the plug or cord with wet hands.\n* Avoid exposing the travel adapter to direct sunlight for extended periods of time.\n* Do not pull or tug on the cord.\n* Keep the travel adapter and its cords away from heat sources such as ovens, stoves, or direct sunlight.\n* Never use this appliance near water sources.\n* Turn off the travel adapter before plugging it into an electrical outlet.\n* Unplug the travel adapter when not in use.\n* Do not use this appliance with an extension cord.\n* Keep the travel adapter and its cords away from children and pets.\n* If any part of the travel adapter or its cords is damaged, discard it and do not use it.\n\n16\n\nFor more information on safety and usage, refer to the user's manual available at supportapj.dell.com.\n\nFeel free to contact Dell Support at 1-800-349-2027 if you have any questions or concerns regarding this travel adapter or your headset.\n\nFor more information on other safety and usage guidelines, refer to the user's manual available at supportapj.dell.com.\n\nUsing Your Mini 3i Smartphone\n\n1. How do I clean my headset?\n\n‘The headset does not fully disassemble. To clean it, use a soft dry cloth to wipe off any dirt or debris from the ear cup and microphone. Avoid using water or other liquids as they can cause damage to the headset.\n\n2. Can other Bluetooth phone users hear my conversation?\n\n‘No. The wireless technology used in your headset is not easily monitored by third parties because Bluetooth wireless signals are significantly lower in radio frequency power than those produced by a typical mobile phone. Adjust the headset volume, or move to another area and try again.\n\n3. The travel adapter and my headset are not properly connected.\n\n‘Separate the travel adapter from the headset, reconnect them, and charge the headset. If the issue persists, contact Dell Support at 1-800-349-2027.\n\n4. How do I change my device's battery?\n\n‘Do not attempt to change your device's battery. If the battery needs to be replaced, contact Dell Support at 1-800-349-2027 for assistance.\n\n5. I dropped my device and its screen is cracked. What can I do?\n\n‘Contact Dell Support at 1-800-349-2027 for repair options or to receive an estimate of repairs.\n\n6. How do I change the wall outlet that my device is charging on?\n\n‘Unplug the travel adapter and your device from the wall outlet, wait for the charge to finish before switching the outlets, and then plug in again.\n\n17\n\nFeel free to contact Dell Support at 1-800-349-2027 if you have any questions or concerns regarding this travel adapter or your headset. For more information on safety and usage, refer to the user's manual available at supportapj.dell.com. ", ' The image shows a smartphone dock interface. On the screen, there is an application called "PhotoFrame" displayed with a slide show of photos on the tablet and media card. The user can touch the screen to access settings for the PhotoFrame application, including adjusting the time interval for the pictures.\nThe text at the bottom of the image indicates that in order to use the smartphone, one must activate it by signing up for a service plan with a service carrier in the area and registering the phone with the network. The battery, SIM card, and microSD card must also be installed and the battery charged before the phone can be used. The image includes a compatibility chart for connecting headphones to the smartphone. ', " This image appears to be a product description or manual for a Samsung device, specifically mentioning that it supports Bluetooth connectivity with Bose devices. The text also provides information on the product's warranty, specifications such as battery life and operating range, and usage instructions such as cleaning, installation, and safety precautions. Additionally, there are technical details about the dimensions and weight of the device, as well as system requirements for use with certain Bose products and compatibility with multi-disc changers. ", ' There are two types of SoundLink adapters: for use with a computer and for use with a Wave radio. The adapter for use with a computer connects directly to the Bose link connector on the rear of the system. To disconnect the system from the computer, press the button on the end of the USB key and then remove the USB key from the computer USB port. If you have problems connecting, see "Troubleshooting" on page 11. The SoundLink remote control gives access to music on the computer and controls all of the system features, including mute, volume, play/pause, seek/track, and scan mode. ', ' ',' 3 Dual Microphone — Enhances sound quality for better audio experience during calls.\n4 Power button — Turns the device on or off.\n5 Volume buttons — Adjust the volume of the phone, media playback, and ringtones.\n6 Home button — Navigates to the home screen and other applications.\n7 Back button — Goes back to the previous screen or menu item.\n8 Camera button — Quickly launches the camera application.\n9 Battery status indicator — Shows battery level and charging status.\n10 Charging port — Connects to a charger for recharging.\n11 Earphone jack — Allows you to connect wired earphones or headsets. ', ' The Mini 3i is designed to connect with other Bose sound systems using the Bose Link connector. It is not compatible with multi-disc changers or Bluetooth devices. For more information about the Bose Link connector, contact the Bose organization in your country.\n\nFor technical support or assistance with a return or shipment, please contact the Bose organization in your country using the phone numbers and website addresses provided on page 15.\n\nThank you for choosing the Mini 3i Music Adapter. We hope you enjoy listening to your music through this innovative product.\n\nBose Corporation\n1601 Technology Way, Framingham, MA 0170-3784\nA Bose® company\nwww.Bose.com\n\nThe Bluetooth trademark is the property of the Bluetooth Special Interest Group (SIG). The Bluetooth word mark and logos are registered trademarks owned by Bluetooth SIG, Inc. and any use of such marks by Bose Corporation is under license. ', ' 1. Keep the travel adapter away from water sources, such as sinks, bathtubs or showers.\n2. Do not immerse the travel adapter in water.\n3. Ensure that the cord does not come into contact with water or other liquids while in use or during normal handling.\n4. Avoid plugging the travel adapter into an electric outlet on a wet surface.\n5. Do not touch the metal parts of the travel adapter while it is plugged into the wall socket.\n6. Keep the travel adapter away from heat sources such as radiators, ovens or hair dryers.\n7. Turn off the power before inserting or removing the plug from the outlet.\n8. Unplug the travel adapter when not in use to reduce the risk of electric shock.\n9. Do not block the air vents on the travel adapter.\n10. Use a grounded electrical outlet for the travel adapter.\n\nThis device was tested and certified by UL. It conforms to UL 285 and Canadian Standards Association (CSA) standards.\n\n* This device should be discarded with other electronic waste. Please follow local recycling rules.\n16\n\n ', ' 2 Touch and drag the slider to set the time interval (in seconds) for the slide show to stop after the last picture. The default time interval is 60 seconds.\n\n[3 Touch and drag the slider to set the brightness of the screen for the PhotoFrame. The default brightness level is 50%.\n\n[4 Touch and drag the slider to set the volume of the music being played during the slide show. The default volume level is 70%.\n\n[5 Click OK to exit the settings interface.\n\n19\n\n  \n\n13\n\n  ', ' connection between your headset and your phone. After pairing, you can wirelessly listen to music or talk on the phone.\n\n1. Turn on your headset by sliding the power switch. The blue indicator light flashes four times.\n2. On your phone, go to Bluetooth settings and turn on Bluetooth.\n3. Select "Bose" from the list of available devices. If prompted, enter the default password 0000.\n4. Your phone will now connect to your headset wirelessly. You can see this connection in the Bluetooth settings menu on your phone.\n\nUsing your headset\n-------------------------\n\n1. Once connected, you can play or pause music or answer/end calls by pressing the center button (B) or the Volume up or down button (A).\n2. To switch between phone and media calls, press and hold the center button (B) for 3 seconds. The LED light will change color to indicate the new connection type.\n3. To redial the last incoming call, press and hold the center button (B) for 5 seconds.\n4. To activate voice commands, press and hold the center button (B) until you hear a beep. Then release the button and speak your command.\n\nCharging and battery life\n-------------------------------\n\n1. Your headset can be charged using the supplied micro-USB cable. When charging is complete, the LED light will change to blue. The battery indicator light will turn off when fully charged.\n2. The battery life of your headset is approximately 6 hours of talk time or up to 8 hours of music playback.\n\nCleaning and maintenance\n---------------------------------\n\n1. Wipe the ear cushions with a soft, dry cloth. For stubborn stains, use mild soap and warm water.\n2. Do not immerse your headset in water.\n3. If the ear cushions become deux yuri\n4. Si 5)1.', " Features\n\n• +Long-clothe remote control\n• Through RS-485 communication, remote control is possible up to 1.2 Kilometers,\n• Easy Control\n• Easy and simple §-way keys for controling the camera.\n\nInstructions\n\n1. To turn on your camera, press the power button located at the top of the controller. The power light will appear when it is turned on.\n\n2. To change the channel, press the channel up or down buttons. The channel display will show the current channel.\n\n3. To adjust the volume, press the volume up or down button. The volume display will show the current volume level.\n\n4. To mute the audio output, press and hold the mute button located on the top left corner of the controller.\n\n5. To exit the settings menu, press the back button located on the top right corner of the controller.\n\n6. To turn off your camera, press and hold the power button until the power light turns off.\n\n7. To access the setup menu, press and hold the Menu button located on the bottom left corner of the controller.\n\n8. To change the input source, press the input source button located on the bottom right corner of the controller. The display will show the current input source.\n\n9. To navigate through the setup menu, use the directional buttons located in the center of the controller and press the OK button to select an option.\n\n10. To exit the setup menu, press the back button located on the top right corner of the controller.\n\n11. For more information on how to use your camera remote controler, refer to the user's manual included with your camera.\n\nNotes:\n\n• For more information on how to connect your camera to a coaxial connection or COAXITRON supporting camera, refer to the user's manual included with your camera.\n\n• Do not block any ventilation openings on your camera or remote controler. Blocking these openings may cause overheating and damage to the device. ", ' 1. Turn on your phone’s Bluetooth function, then search for nearby Bluetooth devices. Your headset should appear in the list of available devices.\n2. Touch the name of your headset to select it, and then touch “Connect” or “Pair” to begin pairing. If prompted, enter the default pairing code (0000) to complete pairing.\n3. Once paired, you can answer incoming calls directly through the headset, or initiate a call by pressing the Talk button on the headset.\n4. During a call, you can use the Volume up and down buttons to adjust the volume, or press the Talk button to end the call.\n5. To disconnect the headset from your phone, simply press and hold the Volume down button until the indicator light flashes red 3 times, then release the button.\n\nUsing your headset with a non-Bluetooth device\n\n1. Connect the headset to the 3.5mm audio jack of your non-Bluetooth device using the provided cable.\n2. Once connected, you can use the Volume up and down buttons to adjust the volume, or press the Talk button to answer incoming calls or initiate a call.\n3. To disconnect the headset from your device, simply unplug the 3.5mm audio jack cable from your device.\n\nTroubleshooting\n\nIf you encounter any issues with your headset, try the following troubleshooting steps:\n\n1. Make sure that your headset and phone or other device are within close range of each other (within 10 feet).\n2. Check to see if either device is running low on battery power. If so, recharge both devices before attempting to use them again.\n3. Try restarting both the headset and the connected device. This can sometimes resolve connectivity issues.\n4. If you are still having trouble connecting your headset to your device, try resetting the Bluetooth settings on both devices.\n5. If none of these troubleshooting steps work, contact Samsung for further assistance. ', " connection between your phone and the headset. To pair and connect the headset, follow these steps:\n\n1. Turn on your phone's Bluetooth function.\n2. On the headset, press and hold the Power button (B) until the blue indicator light flashes 4 times. This indicates that the headset is in discoverable mode.\n3. On your phone, search for available Bluetooth devices. The name of the headset should appear in the list of available devices. Tap on it to initiate pairing.\n4. Follow the prompts on your phone to complete the pairing process. Your phone will now be connected to the headset.\n\nUsing your headset:\n\n1. To answer a call, press and hold the Talk button (A) until you hear the sound of the call.\n2. To end a call, press the Talk button again or hang up on your phone.\n3. To activate voice commands or access other functions, press and hold the Voice Command button (C).\n4. To adjust the volume or skip tracks, use the Volume Up/Down buttons (D) and the Previous/Next Track buttons (E).\n5. When you're finished using your headset, press and hold the Power button (B) until the blue indicator light flashes 4 times to turn it off. ", ' 2 Touch the plus icon to add photos from your tablet or media card to the PhotoFrame slide show.\n[fig in the bottom left corner of the screen.\n\n \n\n19\n\n ', ' Risks OF ELECTRICAL SHOCK: To reduce the risk of electric shock, do not immerse the power adapter in water or other liquids. Keep the power adapter away from water sources such as bathtubs, sinks, and toilets. If the power adapter comes into contact with water, immediately unplug it from the electrical outlet and avoid using it until it has completely dried.\n\nRISKS OF FIRE: Avoid exposing the power adapter to excessive heat, direct sunlight, or flammable materials such as curtains, bedding, and clothing. Keep the area around the power adapter clear of clutter and flammable objects. Unplug the power adapter from the electrical outlet before cleaning it with a damp cloth. Do not use the power adapter if you observe any signs of damage or abnormal smell.\n\nRISKS OF ELECTRIC SHOCK: To reduce the risk of electric shock, do not immerse the headset in water or other liquids. Keep the headset away from water sources such as bathtubs, sinks, and toilets. If the headset comes into contact with water, immediately unplug it from the electrical outlet and avoid using it until it has completely dried.\n\nRISKS OF FIRE: Avoid exposing the headset to excessive heat, direct sunlight, or flammable materials such as curtains, bedding, and clothing. Keep the area around the headset clear of clutter and flammable objects. Unplug the power adapter from the electrical outlet before cleaning it with a damp cloth. Do not use the headset if you observe any signs of damage or abnormal smell.\n\nDO NOT PLUG THE HEADSET INTO A DIFFERENT ELECTRICAL OUTLET OR CHARGER, AND DO NOT TAMPER WITH THE HEADSET OR TRY TO OPEN IT. ', ' The Bose® OE2 audio headphones come with a two-year warranty against defects in materials or workmanship. If you experience any issues with your product within this timeframe, please contact our Customer Service department at 1-800-444-2673 for assistance.\n\nPlease keep in mind that using the headphones outside of their intended purpose—listening to music, watching videos or making phone calls—could result in damage to the product or personal injury due to environmental hazards such as moisture, extreme temperatures or exposure to high noise levels.\n\nThank you for choosing Bose®. We hope you enjoy your new headphones and that they provide many hours of listening pleasure. If you have any further questions or concerns, please do not hesitate to contact us.\n\nSincerely,\nBose Corporation ', ' 3.5mm audio cable and headband are available for purchase at a Bose retailer or online store.\nFor information on replacement parts and accessories, visit www.bose.com/support.\n\n ', ' The image appears to be a scanned or photographed document with safety and usage instructions for an electronic device, specifically a vacuum cleaner. At the top, there is a warning that the appliance is intended for household use only and should not be used in commercial or industrial environments. There are several images and corresponding text boxes providing information on how to use the vacuum cleaner safely and correctly.\n\nOn the left side of the document, there are sections titled "Assembly" and "Disassembly," with accompanying images showing how to assemble and disassemble the vacuum cleaner. The right side includes a section titled "Safety Information" and a table that outlines safety information in various languages.\n\nThe bottom of the document contains contact information for Bose Corporation, with separate phone numbers and websites listed for different countries, including Australia, Belgium, Canada, China, Denmark, Germany, France, India, Ireland, Italy, Japan, Mexico, and others. The text is too small to read in detail, but it seems to provide customer support and product information contact details. ', ' ', ' This is an image of a user manual for a product, specifically a duoStation interface. The interface allows users to connect their tablet to a charging dock with additional features such as displaying photos in a slideshow, adjusting the volume or brightness, and changing settings like the default application launched when the tablet is docked. There are safety and maintenance instructions for the product included in the manual, as well as information on i Certification and Industry Canada certification. The image also contains a disclaimer about not modifying the device and complying with FCC regulations. ', " I'm sorry, but it appears that you have provided an image of a product along with some text that may be related to the product. However, without additional context, it is difficult to determine the exact nature of the image and how it relates to your question. Can you please provide more information or clarify what you are asking? ", " 3. Vacuum cleaner will stop functioning if it is left on for extended periods. For safety reasons, turn vacuum off when not in use.\n\n4. Do not try to exceed the limits of the pivots or over-rotate the earcups.\n\n5. Do not drop, sit on, or allow the headphones to be immersed in water.\n\n6. If you experience a warming sensation or loss of audio when using mobile phone adapters, remove and aislenct immediately.\n\n7. Do not use mobile phone adapters to connect headphones to airplane seat jacks, as this could result in personal injury such as burns or propeity damage due to overheating.\n\n8. Replace the earcup cushions if they become worn or damaged.\n\n9. Always follow the manufacturer's instructions for proper use and maintenance of your vacuum cleaner. ", ' ', ' This is the user manual for a Bose mobile device charger with a docking station. The document is in English and provides instructions on how to use the charger and docking station. It includes information about connecting the equipment, undocking the tablet, using the dock, and troubleshooting issues such as intermittent audio and excessive bass. The manual also includes contact information for Bose Corporation in various countries. ', ' router.\n\n* Ensure that the USB key is connected to your computer.\n\n* Remove the USB key from your computer, wait 30 seconds and insert it again.\n\n* Try restoring the wireless link between the SoundLink™ adapter\nand USB key. See “Manually linking the SoundLink™ adapter and USB key”\non page 15.\n\n* If you are using a desktop computer, try moving the USB key to a port that\nprovides a more direct path to the adapter. Or use a USB extension cable,\navailable for purchase at computer stores.\n\n* Disconnect the power supply from the Wave® system and reconnect it.\n\n ', ' The text provides information about the product and its usage. It includes details such as the warranty, specifications, and instructions for pairing and connecting the headset to a Bluetooth device. There are also answers to some questions that may arise during the use of this product. ',
 ' .Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused.\n\nNote: The serial number is located on the warranty card included in the carton.\n\n16\n',
 ' Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently on the outside of the carton. Cartons not bearing a return authorization number will be refused.\n\nNote: The serial number is located on the warranty card included in the carton.\n\n16\n` ',
 ' 11. Troubleshooting (cont.)\n\nWhat to do\n\nIf you experience any trouble using your headphones, try the following troubleshooting instructions:\n\nSymptom A: SOUNDLINK or COMPUTER never appear on the Wave® system display and no lights blink.\n\n* Ensure that both ends of the audio cable are securely connected.\n* Make sure the computer is turned on and that its volume control is set to maximum.\n* Make sure you have downloaded and installed the SoundLink™ software for your operating system.\n\nIf this does not resolve the problem, contact Bose Customer Service. Refer to the address list included in the carton.\n\n12. Auxiliary Ports (cont.)\n\nWhat to do:\n\nSymptom A:\nSOUNDLINK or COMPUTER never appear on the Wave® system display and no lights blink continuously.\n= NO SIGNAL - appears eventually on the system. display.\n\nOr, see Symptom B on the next page.\n\n* Press and hold the VOLUTE button until the volume level becomes 2 Pp.\n* Ensure that you are using a Wave® USB adapter (included with your kit) to connect your SoundLink™ to the system.\n* Make sure that your computer is turned on and its audio output port is connected to the Wave® music system.\n\nIf this does not resolve the problem, contact Bose Customer Service. Refer to the address list included in the carton.\n\n13. Auxiliary Ports (cont.)\n\nWhat to do:\n\nSymptom B:\nCONNECTING remains on the Wave® system display and the adapter light blink continuously.\n= NO SIGNAL - appears eventually on the system. display.\n\nOr, see Symptom A on the previous page.\n\n* Ensure that you are using a Wave® USB adapter (included with your kit) to connect your SoundLink™ to the system.\n* Make sure that your computer is turned on and its audio output port is connected to the Wave® music system.\n* If you received an upgrade CD with your kit and have not used it, be sure to run it now.\n\nIf this does not resolve the problem, contact Bose Customer Service. Refer to the address list included in the carton.\n\n14. Auxiliary Ports (cont.)\n\nWhat to do:\n\nSymptom B:\nCONNECTING remains on the Wave® system display and the adapter light blink continuously.\n= NO SIGNAL - appears eventually on the system. display.\n\nOr, see Symptom A on the previous page.\n\n* Ensure that you are using a Wave® USB adapter (included with your kit) to connect your SoundLink™ to the system.\n* Make sure that your computer is turned on and its audio output port is connected to the Wave® music system.\n* If you received an upgrade CD with your kit and have not used it, be sure to run it now.\n\nIf this does not resolve the problem, contact Bose Customer Service. Refer to the address list included in the carton.\n\n15. Auxiliary Ports (cont.)\n\nWhat to do:\n\nSymptom B:\nCONNECTING remains on the Wave® system display and the adapter light blink continuously.\n= NO SIGNAL - appears eventually on the system. display.\n\nOr, see Symptom A on the previous page.\n\n* Ensure that you are using a Wave® USB adapter (included with your kit) to connect your SoundLink™ to the system.\n* Make sure thatyour computer is turned on and its audio输出口是连接到theWave®音乐系统。\n* If您收到一个升级的CD，并未使用它，请确保运行它。\n\n如果这样做不能解决问题，请联系Bose客户服务。参考地址表中包含的联系地址列表。 ',
 ' Bose.com/register for Bose contact information in your country/region) for specific return and shipping instructions;\n\n2. Label and ship the product, freight prepaid, to the address provided by the Bose organization in your country; and\n\n3. Place any necessary return authorization number prominently ‘on the outside of the carton. Cartons not bearing a return authorization number, where required, will be refused.\nNote: The serial number is located on the warranty card included in the carton.\n\nThe warranty for this product is valid in countries where Bose products are sold. Any repairs or replacements provided under the limited warranty will be made with new or reconditioned parts as determined by Bose, and any service performed will be done using Bose authorized service personnel or facilities.\n\nThis limited warranty does not cover damage caused by normal wear and tear, misuse, abuse, neglect, modification, accident, improper handling, installation in an unsuitable location, exposure to extreme temperatures, or other acts of God or the public enemy.\n\nBose reserves the right to repair, replace or dispose of any defective product, at its sole discretion, if the above warranty conditions are not met. Any implied warranties are limited in duration to the duration of this limited warranty. Some jurisdictions do not allow limitations on duration of implied warranties so the duration of such warranties may be longer.\n\nThis limited warranty gives you specific legal rights. You may have other rights under the laws of your country or region.\n\n16\n',
 " the computer audio output to be directed to the Wave® system. Check the owner's guide for instructions on how to do this with your particular computer.\n\nIf you are using a Windows computer and need additional assistance, please refer to the Bose® SoundLink™ software download at Global.Bose.com/SoundLink. ",
 " Please note that the above instructions are for a specific product and may not apply to all Bose products. Always follow the manufacturer's instructions for your specific product. ",
 ' The Bose Powerfoot Bagless Vacuum Cleaner is designed to provide you with the best possible cleaning experience. However, as with any vacuum cleaner, normal wear and tear can occur over time. Here are some common issues that may arise and what you can do to resolve them:\n\n1. No suction or audio in one earcup only: Ensure that both ends of the audio cable are securely connected. If this persists, try a different audio device.\n2. Distorted bass: Make sure both earcup cushions are firmly seated. The high-density foam will regain its shape after a few minutes in a neutral position.\n3. Crack or hole in the Hose: Broken or worn Drive Belt - Replace Drive Belt.\n4. Rotating Floor Brush bristles worn: Replace Brush.\n5. Rotating Floor Brush jammed: Clear clog, see page 8-9.\n6. Clog in vacuum: Empty Dust Bin.\n7. Dust Bin full: Empty Dust Bin.\n8. Dust Bin not installed properly: Position correctly and lock in place. Check Pre-motor and Post-motor filters.\n9. Visible dirt escaping from cleaner: Position correctly and lock in place. Check Pre-motor and Post-motor filters. Maintaining Mini Turbo Brush, see page 11.\n\nIf any problem persists, contact Bose Customer Service for return and shipping instructions. Please refer to the warranty card included in the carton for details on how to register your product. Failure to register will not affect your limited warranty rights. The serial number is located on the warranty card. ',
 ' ',
 " Sonvnanal Nivin £0.25\n\nmaintenance\\_12\n\nTroubleshooting\n\nCheck this list of possible problems and solutions before bringing your vacuum to a service center.\n\n13,\n\nPROBLEM\nVacuum cleaner won't run\n\n* Power cord not plugged in\n* Blown fuse/tripped breaker in home electrical system\n\npossible solution\n\n* Check electrical plug\n* Replace fuse or reset breaker\n\n‘Vacuum cleaner and tools won't pick up dirt\n\n* Incorrect Height Adjustment\n\n* Hose not attached to suction opening\n\n* Twist Snap Hose not securely attached\n\n* Crack or hole in Hose\n\n* Broken or worn Drive Belt\n\n* Rotating Floor Brush bristles worn\n\n* Rotating Floor Brush jammed\n\n* Clog in vacuum\n\n* Dust Bin full\n\n* Dust Bin not installed properly\n\n* Filters are dirty\n\n14,\nMini Turbo Brush is clogged or debris is wrapped around the brush roll\n\npossible solution\n\n* Cut off pet hairs or threads surrounding brush with scissors\n\n15,\nVisible dirt escaping from cleaner\n\npossible solution\n\n* Dust Bin full\n\n+ Filters missing or installed incorrectly\n\n "]

In [ ]:
print(len(answers))

15


In [ ]:
final = saved + answers9

In [ ]:
print(len(final))

90


In [ ]:
ground_truth = df['ground_truth'].tolist()
ground_truths = ground_truth[:90]
ground_truths = [[value] for value in ground_truths]
print(len(ground_truths))

90


In [ ]:
data = {
    "question": questions[:90],
    "answer": final,
    "ground_truths": ground_truths,
    # "contexts": contexts,
}

In [ ]:
# convert dict to dataset
dataset = Dataset.from_dict(data)
print(dataset)

Dataset({
    features: ['question', 'answer', 'ground_truths'],
    num_rows: 90
})


In [ ]:
print(answers[1:])

[]


In [25]:
final = answers+ answers2

In [26]:
from bert_score import score

# Example texts
generated = final

# Calculate BERTScore
P, R, F1 = score(generated, ground_truths, lang='en', verbose=True)

# Print scores
print(f"Precision: {P.mean()}")
print(f"Recall: {R.mean()}")
print(f"F1 Score: {F1.mean()}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 1.67 seconds, 59.71 sentences/sec
Precision: 0.8454497456550598
Recall: 0.8923017978668213
F1 Score: 0.8680036664009094
